In [1]:
# This code is a recreation of the original tube making code in Python. 
# The input is a magno video and the output is a list of tubes and an output video with tubes overlaid.
# Here, we break the output up into "stages," which build upon each other 


# Necessary inputs
import cv2
import numpy as np
import argparse
import csv
import os
import math
import random
import time
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D 
from sklearn.cluster import DBSCAN
from copy import deepcopy
from operator import itemgetter
from itertools import *


In [2]:
# Useful Functions
def make_ellipse(image, row):
    center = (int(round(row[0])),int(round(row[1])))
    axes = (int(round(row[2])*2 ),int(round(row[3])*2) )
    angle = int(round(row[4]))
    color = (row[8],row[9],row[10])
    thickness = 4  #Do -1 if you want it filled in

    return cv2.ellipse(image, center, axes, angle, 0, 360, color, thickness) 

def make_ellipse2(image, ellipse):
    thickness = 4  #Do -1 if you want it filled in
    
    return cv2.ellipse(image, ellipse[1], ellipse[3], thickness) 

def make_ellipse3(image, ellipse,tubenum):
    thickness = 4  #Do -1 if you want it filled in
    center = (int(ellipse[0][1][0][0]), int(ellipse[0][1][0][1]) )
    
    cv2.putText(image,str(tubenum), center,cv2.FONT_HERSHEY_PLAIN, 2, (255,255,255), thickness)
    return cv2.ellipse(image, ellipse[0][1], ellipse[0][3], thickness) 

def draw_event(image, event):
    thickness = 4
    center = (int(event[2]), int(event[4]) )
    
    return cv2.putText(image,"X", center,cv2.FONT_HERSHEY_PLAIN, 4, (0,0,255), thickness)

def writeTubelist(tubelistLoc, tubes):
    with open(tubelistLoc, 'w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(["f","x","y","b","a","phi","Vx","Vy","R","G","B","T"])

        for tube in tubes:
            for ell in tube:
                writer.writerow([ell[0],ell[1][0][0],ell[1][0][1],ell[1][1][0],ell[1][1][1],ell[1][2],ell[2][0],ell[2][1],ell[3][0],ell[3][1],ell[3][2],tubes.index(tube) ])

def readTubelist(tubelistLoc):
        with open(tubelistLoc) as csvfile:
            readCSV = csv.reader(csvfile, delimiter=',')
            next(readCSV) #burn the first line
            tubes2 = []
            
            for row in readCSV:
                row = [float(i) for i in row] #text-->float for each thingy
                tubes2.append(row) #Read into intermediate var
                
        numtubes = tubes2[-1][-1]+1
        tubes = []
        for i in range(0,int(numtubes)):
            thistube = [ [e[0],((e[1],e[2]),(e[3],e[4]),e[5]),[e[6],e[7]],[e[8],e[9],e[10]] ] for e in tubes2 if e[-1]==i]
            tubes.append(thistube)
        
        return tubes
                

def getBoxFromEllipse(e):
    # Bounding box is normally (xmin, ymin, width, height) 
    bbox = [int(e[1][0][0]-e[1][1][0]/2), int(e[1][0][1] - e[1][1][1]/2), int(e[1][1][0]), int(e[1][1][1]) ]
    if bbox[0]<0:
        bbox[0]=1
    if bbox[1]<0:
        bbox[1]=1
    return tuple(bbox)

def getEllipseFromBox(b, rot=0):
    # Ellipse is ((centerx, centery), (smallax, bigax), degrotation)
    # maybe pass in rotation
    center = (b[0]+b[2]/2 , b[1]+b[3]/2)
    size = (b[2], b[3])
    ell = (center, size, rot)
    
    return ell

def writevid(vid, outvidpath):
     #Look at imageshape output and take size from there (reversed)
    H = vid[1].shape[0]
    W = vid[1].shape[1]
    numframes = len(vid)
    vidwriter = cv2.VideoWriter(outvidpath, cv2.VideoWriter_fourcc(*'MP4V'),10, (W, H))

    for i in range(1, numframes):
        if i%50==0:
            print("FRAME" , i, "/", numframes)        
        vidwriter.write(vid[i])
               
    vidwriter.release()
    cv2.destroyAllWindows()
    
def readvid(vidinfile):
    #Read in video at vidinfile
    vidcap = cv2.VideoCapture(vidinfile) 
    thevid = []
    F=0
    while(vidcap.isOpened()):
        ret, frame = vidcap.read()
        if ret == False:
            break
        else:
            F=F+1
            thevid.append(frame)

    vidcap.release()
    cv2.destroyAllWindows()
    print(type(thevid))
    return thevid

def putTracksOnVid(tracks, vidinfile, vidoutfile):

    vidcap = cv2.VideoCapture(vidinfile) 
    inputvid = []
    F=0
    
    #Read in video at vidinfile
    while(vidcap.isOpened()):
        ret, frame = vidcap.read()
        if ret == False:
            break
        else:
            F=F+1
            inputvid.append(frame)

    vidcap.release()
    cv2.destroyAllWindows()

    #Put the red rectangles on the vid for the tracks
    outvid = deepcopy(inputvid)
    for f in range(1,len(outvid)):
        here = []
        for T in tracks:
            here.extend([t for t in T if t[0]==f])
        if here: #if any track has a box on this frame
            for h in here:
                bbox = h[1]
                p1 = (int(bbox[0]), int(bbox[1]))
                p2 = (int(bbox[0] + bbox[2]), int(bbox[1] + bbox[3]))
                cv2.rectangle(outvid[f], p1, p2, (0,0,255), 2, 1) #red
            
    #Writes the output vid to a video file
    vidwriter = cv2.VideoWriter(vidoutfile, cv2.VideoWriter_fourcc(*'MP4V'),30, (W, H))
    for i in range(1, F):
        if i%100==0:
            print("FRAME" , i, "/", F)        
        vidwriter.write(outvid[i])
    vidwriter.release()
    cv2.destroyAllWindows()
    
def putMarksOnVid(marks, vidinfile, vidoutfile):

    vidcap = cv2.VideoCapture(vidinfile) 
    inputvid = []
    F=0
    
    #Read in video at vidinfile
    while(vidcap.isOpened()):
        ret, frame = vidcap.read()
        if ret == False:
            break
        else:
            F=F+1
            inputvid.append(frame)

    vidcap.release()
    cv2.destroyAllWindows()

    #Check score for mark and put proper thing on the vid 
    outvid = deepcopy(inputvid)
    for f in range(1,len(outvid)):
        here = []
        for T in marks:
            here.extend([t for t in T if t[0]==f])
        if here: #if any track has a box on this frame
            for h in here:
                bbox = h[1]
                p1 = (int(bbox[0]), int(bbox[1]))
                p2 = (int(bbox[0] + bbox[2]), int(bbox[1] + bbox[3]))
                if h[2]==1:
                    #cv2.rectangle(outvid[f], p1, p2, (0,255,255), 2, 6) #yellow
                    continue
                elif h[2]==2:
                    cv2.rectangle(outvid[f], p1, p2, (0,0,255), 2, 10) #red
                else:
                    cv2.rectangle(outvid[f], p1, p2, (0,255,0), 2, 10) #green
            
    #Writes the output vid to a video file
    vidwriter = cv2.VideoWriter(vidoutfile, cv2.VideoWriter_fourcc(*'MP4V'),30, (W, H))
    for i in range(1, F):
        if i%100==0:
            print("FRAME" , i, "/", F)        
        vidwriter.write(outvid[i])
    vidwriter.release()
    cv2.destroyAllWindows()
    
    
def putTubesOnVid(tubevar, vidoutfile, withnums = False):
    
    if not withnums:
        #Draw the ellipses on the video to check...
        outvid = deepcopy(invid)
        for f in range(1,F):
            puthere = []

            for path in tubevar:
                P = [e for e in path if e[0]==f]
                puthere.extend(P)

            for ellipse in puthere:
                make_ellipse2(outvid[f],ellipse)

        #Writes the thresholded magno vid to a video file
        vidwriter = cv2.VideoWriter(vidoutfile, cv2.VideoWriter_fourcc(*'MP4V'),30, (W, H))
        for i in range(1, F):
            if i%100==0:
                print("FRAME" , i, "/", F)        
            vidwriter.write(outvid[i])
        vidwriter.release()
        cv2.destroyAllWindows()
    
    else:        
        #WITH NUMS    
        #Draw the ellipses on the video to check...
        outvid = deepcopy(invid)
        for f in range(1,F):
            puthere = []

            for path in tubevar:
                P = [e for e in path if e[0]==f]
                T = [tubevar.index(path) for e in path if e[0]==f]
                puthere.append([P,T])

            for [ellipse,Tnum] in puthere:
                if ellipse:
                    make_ellipse3(outvid[f],ellipse,Tnum)


        #Writes the thresholded magno vid to a video file
        vidwriter = cv2.VideoWriter(vidoutfile, cv2.VideoWriter_fourcc(*'MP4V'),30, (W, H))
        for i in range(1, F):
            if i%100==0:
                print("FRAME" , i, "/", F)        
            vidwriter.write(outvid[i])
        vidwriter.release()
        cv2.destroyAllWindows()


def putSomeTubesOnVid(tubevar, vidoutfile, which =[]):
    
    #WITH NUMS    
    #Draw the ellipses on the video to check...
    outvid = deepcopy(invid)
    selectedtubes = [tubevar[w] for w in which]
    
    for f in range(1,F):
        puthere = []

        for path in selectedtubes:
            P = [e for e in path if e[0]==f]
            T = [tubevar.index(path) for e in path if e[0]==f]
            puthere.append([P,T])

        for [ellipse,Tnum] in puthere:
            if ellipse:
                make_ellipse3(outvid[f],ellipse,Tnum)


    #Writes the thresholded magno vid to a video file
    vidwriter = cv2.VideoWriter(vidoutfile, cv2.VideoWriter_fourcc(*'MP4V'),30, (W, H))
    for i in range(1, F):
        if i%100==0:
            print("FRAME" , i, "/", F)        
        vidwriter.write(outvid[i])
    vidwriter.release()
    cv2.destroyAllWindows()

def putEventsOnVid(eventvar, vidinfile, vidoutfile):

    vidcap = cv2.VideoCapture(vidinfile) 
    inputvid = []
    F=0
    
    #Read in video at vidinfile
    while(vidcap.isOpened()):
        ret, frame = vidcap.read()
        if ret == False:
            break
        else:
            F=F+1
            inputvid.append(frame)

    vidcap.release()
    cv2.destroyAllWindows()

    #Draw the events on the video to get a visual
    outvid = deepcopy(inputvid)
    for f in range(1,F):
        puthere = [e for e in eventvar if (f>=e[0]) and (f<=e[1])]
        for event in puthere:
            draw_event(outvid[f],event)

    #Writes the output vid to a video file
    vidwriter = cv2.VideoWriter(vidoutfile, cv2.VideoWriter_fourcc(*'MP4V'),30, (W, H))
    for i in range(1, F):
        if i%100==0:
            print("FRAME" , i, "/", F)        
        vidwriter.write(outvid[i])
    vidwriter.release()
    cv2.destroyAllWindows()


In [36]:
# Stage 0: This stage shows the ellipses before any temporal associations are made. 
# We already filter out the smallest (<20 pix) ellipses before including them in our eList

vidbase = 'ucr3'
inmagnovid = vidbase + '_magno.mp4'

V = readvid(vidbase+".mp4")
vidcap = cv2.VideoCapture(inmagnovid) 
kernel = np.ones((10,10) , np.uint8)
magnovid = []
H=0
W=0
F=0

#Thresholds (at 50) the input magno video into a binary video 'magnovid'
while(vidcap.isOpened()):
    ret, frame = vidcap.read()
    if ret == False:
        break
    else:
        (dontcare, outframe) = cv2.threshold(frame, 50, 255, cv2.THRESH_BINARY)
        cv2.dilate(outframe,kernel, iterations =1)
        cv2.erode(outframe,kernel, iterations =1)
        H = frame.shape[0]
        W = frame.shape[1]
        F=F+1
        magnovid.append(outframe)

vidcap.release()
cv2.destroyAllWindows()


##Write the input video into a variable
invideo = vidbase + '.mp4'
vidcap = cv2.VideoCapture(invideo) 
invid = []
F2 = 0

while(vidcap.isOpened()):
    ret, frame = vidcap.read()
    if ret == False:
        break
    else:
        F2=F2+1
        invid.append(frame)
        
vidcap.release()
cv2.destroyAllWindows()
F = min(F,F2)

#Writes the thresholded magno vid to a video file
vidwriter = cv2.VideoWriter(vidbase +'_magnoOut.mp4', cv2.VideoWriter_fourcc(*'MP4V'),30, (W, H))
for i in range(1, F):
    if i%100==0:
        print("FRAME" , i, "/", F)        
    vidwriter.write(magnovid[i])
vidwriter.release()
cv2.destroyAllWindows()


#Each ellipse is put in the list as a tuple... (framenum, ellipse)
#Ellipse: (x,y) , (MAJORAXIS,minoraxis), angle
eList = []
for f in range(1,F): #for each frame
    M = cv2.cvtColor(magnovid[f], cv2.COLOR_BGR2GRAY);
    _, contours, hierarchy = cv2.findContours(M, 1, 2)
    
    for c in contours:
        if len(c) < 5:
            continue
        area = cv2.contourArea(c)
        if area < 100:
            continue
    
        ellipse = cv2.fitEllipse(c)
        eList.append( [f, ellipse, [0,0], [255,255,255] ] )
        
print(len(eList))
MAXS = np.asarray([max(e[1][1][0],e[1][1][1]) for e in eList if e[0] > 10])
MINS = np.asarray([min(e[1][1][0],e[1][1][1]) for e in eList if e[0] > 10])
AVGS = np.asarray([(e[1][1][0]+e[1][1][1])/2  for e in eList if e[0] > 10])

#Filtering the smallest ellipses (minsize for either dimension = 20pixels)



minsize =  .30 * np.mean(AVGS)
#minsize = 10
eList = [e for e in eList if (e[1][1][0] > minsize) and (e[1][1][1]>minsize)]



#Draw the ellipses on the video just to check...
magnotubes = deepcopy(V)
for f in range(1,F):
    puthere = [e for e in eList if e[0]==f]
    
    for ellipse in puthere:
        make_ellipse2(magnotubes[f],ellipse)
        

#Writes the vid with tubes to a video file
vidwriter = cv2.VideoWriter(vidbase+'_outstage0NEWMINSIZE.mp4', cv2.VideoWriter_fourcc(*'MP4V'),30, (W, H))
for i in range(1, F):
    if i%100==0:
        print("FRAME" , i, "/", F)        
    vidwriter.write(magnotubes[i])
vidwriter.release()
cv2.destroyAllWindows()

<class 'list'>
FRAME 100 / 1184
FRAME 200 / 1184
FRAME 300 / 1184
FRAME 400 / 1184
FRAME 500 / 1184
FRAME 600 / 1184
FRAME 700 / 1184
FRAME 800 / 1184
FRAME 900 / 1184
FRAME 1000 / 1184
FRAME 1100 / 1184
11978
FRAME 100 / 1184
FRAME 200 / 1184
FRAME 300 / 1184
FRAME 400 / 1184
FRAME 500 / 1184
FRAME 600 / 1184
FRAME 700 / 1184
FRAME 800 / 1184
FRAME 900 / 1184
FRAME 1000 / 1184
FRAME 1100 / 1184


In [37]:
print("H:", H)
print("W:", W)
print("F:", F)

H: 720
W: 1280
F: 1184


In [38]:
# Stage 1: We make the first swipe at temporal associations, turning ellipses into paths.

#Doing the first temporal associations. Should start to look like tubes
initialList = []
finalList = []
paths = [[]]
istart = 6
iend = istart+4
fstart = 11
fend = fstart+5
locthresh = 80
sizethresh = 50

while (fend < F):
    initialList = [e for e in eList if e[0]>=istart and e[0]<=iend]
    finalList =  [e for e in eList if e[0]>=fstart and e[0]<=fend]
    
    for ei in initialList:
        for ef in finalList:
            
            matchFlag = False
            r = math.sqrt((ef[1][0][0] - ei[1][0][0])**2 + (ef[1][0][1] - ei[1][0][1])**2 )
            sizediff = ef[1][1][1] - ei[1][1][1] #diff in major axis
            framediff = ef[0]-ei[0]
            
            if not(ei[2]==(0,0)):
                predX = ei[2][0]*framediff + ei[1][0][0]
                predY = ei[2][1]*framediff + ei[1][0][1]
                if (abs(predX-ef[1][0][0]) < 50) and (abs(predY-ef[1][0][1]) < 50):
                    matchFlag = True
            else:
                if (abs(sizediff)<sizethresh) and (r<locthresh):
                    matchFlag = True
            
            if (matchFlag):
                if ei[3]==[255,255,255]:
                    ei[3][0] = random.randint(0,255)
                    ei[3][1] = random.randint(0,255)
                    ei[3][2] = random.randint(0,255)
                ef[3][0] = ei[3][0]
                ef[3][1] = ei[3][1]
                ef[3][2] = ei[3][2]
                
                ef[2][0] = (ef[1][0][0]-ei[1][0][0])/framediff
                ef[2][1] = (ef[1][0][1]-ei[1][0][1])/framediff
                

                #This part searches for ei in existing paths and attaches ef if it finds it
                whichpath = 0
                for i in range(1,len(paths)): 
                    for ell in paths[i]:
                        if ei == ell:
                            whichpath = i
                            
                if not(whichpath==0):
                     paths[whichpath].append(ef)
                else:
                    newpath = []
                    newpath.append(ei)
                    newpath.append(ef)
                    paths.append(newpath)

    #print(fend)
    istart=istart+5
    iend=iend+5
    fstart=fstart+5
    fend=fend+5


    
stage1tubes = paths 
putTubesOnVid(paths, vidbase+'_outstage1.mp4')


FRAME 100 / 1184
FRAME 200 / 1184
FRAME 300 / 1184
FRAME 400 / 1184
FRAME 500 / 1184
FRAME 600 / 1184
FRAME 700 / 1184
FRAME 800 / 1184
FRAME 900 / 1184
FRAME 1000 / 1184
FRAME 1100 / 1184


In [40]:
# Stage 2: If we have 2+ ellipses in the same path on the same frame, chose the larger one

#Fixes the paths so that each color corresponds to a path
newpaths=[[]]

for i in range(1,len(paths)):
    for e in paths[i]:
        RGB = e[3]
        makePathFlag = True
        
        for newpath in newpaths:
            if not newpath or not(newpath[0][3] == RGB):
                continue
            else:
                newpath.append(e)
                makePathFlag = False
                break
        
        if (makePathFlag):
            E = [e]
            newpaths.append(E)

            
#Gets rid of duplicates in each path (took forever smh)
K = deepcopy(newpaths)
newpaths = []

for i in range(0,len(K)):
    bigA = []
    for j in range(0,len(K[i])):
        thise = K[i][j]
        A = [K[i].index(e) for e in K[i] if e==thise]
        #print("A:", A)
        bigA.extend(A)
    
    out = [K[i][t] for t in list(set(bigA))]
    newpaths.append(out)

#Make sure newpaths is in frame order
for P in newpaths:
    P.sort(key = lambda x: x[0])
    


#Handles when there are 2 ellipses from the same tube on a single frame.
L = deepcopy(newpaths)

#toRemove = []

for i in range(0,len(L)):
    tubeSplits = []
    R =0.5*np.mean(AVGS)
        
    for f in range(1,F):
        A = [e for e in L[i] if e[0] == f] #list of ellipses on this frame
        if len(A)>1:
            A.sort(key = lambda x: x[1][1][0]**2 + x[1][1][1]**2) 
            distList = [math.sqrt((a[1][0][0]-b[1][0][0])**2 + (a[1][0][1] - b[1][0][1])**2) for a in A for b in A if a!=b]
            if all(d>R for d in distList): #All the ellipses are far enough apart
                print("frame", f, "List", A)
                if not(tubeSplits):
                    tubeSplits = [[k] for k in A]
                    #collf = tubeSplits[0][0][0]
                    #toRemove.append([i,collf])
                else:
                    #ADD THINGS IN A TO TUBESPLITS ( [[e], [e]] ) APPROPRIATELY        
                    for a in A:
                        #check each distance and pick the smallest
                        dists = [math.sqrt((a[1][0][0]-tubeSplits[k][-1][1][0][0])**2 + (a[1][0][1] - tubeSplits[k][-1][1][0][1])**2) for k in range(0,len(tubeSplits))]
                        tubeSplits[dists.index(min(dists))].append(a)   
            
            else:
                L[i] = [e for e in L[i] if e not in A[:-1]] #get rid of the smaller one(s)
                
                
    #at the end of the tube
    if tubeSplits:
        if len(tubeSplits[0]) > 5 :
            print("Tube", i,"is getting snipped. TS length = ",len(tubeSplits[0]))
            #L[i] = [e for e in L[i] if e[0] < tubeSplits[0][0][0]]
            #IF SOMETHING IS BROKEN IT'S THIS LINE RIGHT UP HERE^^^ 

            for T in tubeSplits:
                RR = random.randint(0,255)
                G = random.randint(0,255)
                B = random.randint(0,255)
                for e in T:
                    e[3][0] = RR
                    e[3][1] = G
                    e[3][2] = B
                L.append(T)

#for tr in toRemove:
#    print("Deleting from", tr[0], "starting at ", tr[1])
#    print("Before", len(L[tr[0]]))
#    L[tr[0]] = [ell for ell in L[tr[0]] if ell[0] < tr[1]]
#    print("After", len(L[tr[0]]))
    
newpaths = L
stage2tubes = newpaths
putTubesOnVid(newpaths,vidbase+ '_outstage2.mp4')


frame 53 List [[53, ((377.592041015625, 482.4469299316406), (23.11284065246582, 38.023780822753906), 116.14979553222656), [17.087890625, -11.981557210286459], [87, 128, 12]], [53, ((329.37896728515625, 513.4828491210938), (48.356021881103516, 106.30998992919922), 3.9567530155181885), [1.0168660481770833, -1.6362508138020833], [87, 128, 12]]]
frame 54 List [[54, ((376.18231201171875, 481.1770324707031), (21.246036529541016, 37.817142486572266), 121.61817932128906), [12.463485717773438, -9.303642272949219], [87, 128, 12]], [54, ((328.9786682128906, 515.7562255859375), (46.435333251953125, 100.9770278930664), 179.7556915283203), [0.6625747680664062, -0.658843994140625], [87, 128, 12]]]
frame 55 List [[55, ((379.0262451171875, 484.1712341308594), (21.501523971557617, 27.48880958557129), 112.53941345214844), [10.5395751953125, -6.844073486328125], [87, 128, 12]], [55, ((330.0838317871094, 512.8502197265625), (46.96024703979492, 103.45574951171875), 1.7935621738433838), [0.751092529296875, -

frame 540 List [[540, ((344.32366943359375, 398.3611145019531), (21.24561882019043, 32.3521842956543), 113.74835968017578), [-8.291546630859376, 0.672723388671875], [87, 128, 12]], [540, ((401.9166259765625, 395.0621643066406), (36.72731399536133, 60.23565673828125), 80.90355682373047), [3.227044677734375, 0.012933349609375], [87, 128, 12]], [540, ((360.79327392578125, 475.7666015625), (29.053842544555664, 90.68025970458984), 107.34791564941406), [-1.7031982421875, -0.709075927734375], [87, 128, 12]]]
frame 541 List [[541, ((343.21514892578125, 397.9023742675781), (20.05118751525879, 31.134902954101562), 105.67769622802734), [-1.1085205078125, -0.458740234375], [87, 128, 12]], [541, ((397.6094055175781, 394.89459228515625), (37.479454040527344, 59.0704460144043), 82.9105224609375), [-4.307220458984375, -0.167572021484375], [87, 128, 12]], [541, ((359.0849304199219, 474.8482666015625), (29.939159393310547, 93.62483978271484), 108.4199447631836), [-1.708343505859375, -0.9183349609375], [

frame 807 List [[807, ((319.68267822265625, 461.6712951660156), (12.537816047668457, 16.444868087768555), 178.4210662841797), [-8.979248046875, 10.880416870117188], [87, 128, 12]], [807, ((379.6684265136719, 481.21246337890625), (25.50763511657715, 34.27865982055664), 106.22262573242188), [3.34429931640625, 0.7070159912109375], [87, 128, 12]], [807, ((352.7432556152344, 402.9244079589844), (37.287628173828125, 46.905452728271484), 111.00331115722656), [-13.947860717773438, 11.99114990234375], [87, 128, 12]], [807, ((395.0584716796875, 374.2047119140625), (49.61525344848633, 99.91032409667969), 140.4960479736328), [7.209747314453125, -2.3686981201171875], [87, 128, 12]]]
frame 808 List [[808, ((321.90045166015625, 462.9958801269531), (14.117281913757324, 22.055932998657227), 121.623779296875), [-5.246907552083333, 7.695139567057292], [87, 128, 12]], [808, ((379.35546875, 481.1511535644531), (25.13899040222168, 34.94768524169922), 107.988037109375), [2.125213623046875, 0.4509073893229167

frame 31 List [[31, ((346.4054870605469, 402.90643310546875), (14.388435363769531, 15.657615661621094), 114.40092468261719), [2.1500244140625, 0.814971923828125], [95, 110, 252]], [31, ((379.90704345703125, 485.126953125), (18.762771606445312, 23.976655960083008), 122.57814025878906), [1.84185791015625, 0.557861328125], [95, 110, 252]], [31, ((310.3387756347656, 545.687744140625), (46.88889694213867, 104.15167236328125), 1.317153811454773), [0.92138671875, -2.4188232421875], [95, 110, 252]]]
frame 32 List [[32, ((380.6708679199219, 487.0050964355469), (15.077385902404785, 21.96241569519043), 113.31768798828125), [1.3028411865234375, 1.2180023193359375], [95, 110, 252]], [32, ((311.6026306152344, 544.2965698242188), (46.52452087402344, 103.86358642578125), 1.1983039379119873), [1.092620849609375, -1.904998779296875], [95, 110, 252]]]
frame 33 List [[33, ((379.4299621582031, 486.0880126953125), (12.587821006774902, 15.618322372436523), 79.58836364746094), [0.454925537109375, 0.5063069661

frame 446 List [[446, ((652.0018920898438, 140.64698791503906), (28.621585845947266, 47.486454010009766), 7.939383506774902), [0.9464111328125, -0.31787109375], [129, 44, 58]], [446, ((569.9398803710938, 163.6643829345703), (28.72404670715332, 51.40951919555664), 7.610712051391602), [-0.67498779296875, 0.06890869140625], [129, 44, 58]]]
frame 447 List [[447, ((652.9723510742188, 140.58419799804688), (27.459630966186523, 48.69990158081055), 6.5873212814331055), [0.95843505859375, -0.19033050537109375], [129, 44, 58]], [447, ((569.4473266601562, 163.79721069335938), (29.075265884399414, 50.89935302734375), 8.65516471862793), [-0.583770751953125, 0.10086822509765625], [129, 44, 58]]]
frame 448 List [[448, ((654.115234375, 140.16395568847656), (27.090639114379883, 48.49980545043945), 6.636216640472412), [1.0199178059895833, -0.2669677734375], [129, 44, 58]], [448, ((568.7545776367188, 163.88975524902344), (28.83907699584961, 50.8240966796875), 8.355379104614258), [-0.6200968424479166, 0.09

frame 401 List [[401, ((1010.3944702148438, 226.08938598632812), (18.39558219909668, 20.87262535095215), 85.7950210571289), [14.610107421875, -31.216339111328125], [114, 231, 157]], [401, ((995.676025390625, 255.08473205566406), (13.0428466796875, 28.705936431884766), 134.19442749023438), [-0.10833740234375, -2.2209930419921875], [114, 231, 157]]]
frame 426 List [[426, ((957.45166015625, 242.24249267578125), (16.05082893371582, 16.661399841308594), 105.52813720703125), [-41.8563232421875, -1.8094329833984375], [142, 70, 136]], [426, ((999.01171875, 247.3373260498047), (27.673324584960938, 46.17643737792969), 31.087928771972656), [-0.2962646484375, 3.285400390625], [142, 70, 136]]]
frame 427 List [[427, ((956.2410888671875, 243.56813049316406), (15.199714660644531, 18.820377349853516), 83.73843383789062), [-21.533447265625, -0.2418975830078125], [142, 70, 136]], [427, ((994.5067749023438, 252.70565795898438), (19.45555877685547, 31.290002822875977), 161.878662109375), [-2.40060424804687

frame 629 List [[629, ((709.2816162109375, 8.089343070983887), (13.77052116394043, 21.9741153717041), 69.95149993896484), [9.890457153320312, -2.29671311378479], [57, 27, 244]], [629, ((668.21630859375, 17.19430923461914), (18.77276611328125, 26.703956604003906), 48.985755920410156), [-0.3758697509765625, -0.020471572875976562], [57, 27, 244]]]
frame 630 List [[630, ((710.3106689453125, 7.257750511169434), (11.760616302490234, 18.680709838867188), 62.12818145751953), [8.11817626953125, -2.003689002990723], [57, 27, 244]], [630, ((667.0879516601562, 17.87327766418457), (17.557119369506836, 23.118343353271484), 38.552181243896484), [-0.5263671875, 0.11941642761230468], [57, 27, 244]]]
frame 631 List [[631, ((758.0701293945312, 33.013587951660156), (13.600847244262695, 20.838115692138672), 74.18388366699219), [47.75946044921875, 25.755837440490723], [57, 27, 244]], [631, ((667.0036010742188, 18.285078048706055), (16.450326919555664, 18.793466567993164), 51.40932083129883), [-0.08435058593

frame 897 List [[897, ((931.7771606445312, 272.096923828125), (22.23655128479004, 24.766185760498047), 148.16921997070312), [-3.923004150390625, 11.885162353515625], [23, 53, 225]], [897, ((944.1702880859375, 242.44190979003906), (33.12826156616211, 63.7452278137207), 130.67483520507812), [2.2735595703125, -2.9423446655273438], [23, 53, 225]]]
frame 898 List [[898, ((962.9019165039062, 260.7526550292969), (19.651708602905273, 22.551790237426758), 111.73747253417969), [7.75958251953125, 4.142018636067708], [23, 53, 225]], [898, ((931.9253540039062, 272.5979919433594), (22.577478408813477, 26.837594985961914), 153.4136199951172), [-2.5659383138020835, 8.090464274088541], [23, 53, 225]], [898, ((936.6890258789062, 236.57862854003906), (34.26142501831055, 35.57740783691406), 173.60862731933594), [-0.9780476888020834, -3.915990193684896], [23, 53, 225]]]
frame 922 List [[922, ((847.3208618164062, 297.63909912109375), (11.9801025390625, 18.465978622436523), 86.14019775390625), [-29.077026367

frame 1070 List [[1070, ((342.0654602050781, 396.5195617675781), (15.86661434173584, 17.410255432128906), 158.02133178710938), [-5.831888834635417, 0.06844075520833333], [51, 112, 189]], [1070, ((402.3997802734375, 385.28680419921875), (20.108407974243164, 26.85540771484375), 161.99913024902344), [0.518145751953125, -1.546533203125], [51, 112, 189]]]
frame 1071 List [[1071, ((342.52703857421875, 396.6375427246094), (16.584697723388672, 18.92909812927246), 140.79042053222656), [0.461578369140625, 0.11798095703125], [51, 112, 189]], [1071, ((402.7608642578125, 385.3798522949219), (21.85226821899414, 27.857667922973633), 140.6716766357422), [0.361083984375, 0.093048095703125], [51, 112, 189]]]
frame 1072 List [[1072, ((342.12506103515625, 397.0047607421875), (18.474227905273438, 21.59659194946289), 125.483642578125), [0.0298004150390625, 0.2425994873046875], [51, 112, 189]], [1072, ((402.3869323730469, 386.4310607910156), (24.572650909423828, 26.836091995239258), 169.72421264648438), [-0.

FRAME 100 / 1184
FRAME 200 / 1184
FRAME 300 / 1184
FRAME 400 / 1184
FRAME 500 / 1184
FRAME 600 / 1184
FRAME 700 / 1184
FRAME 800 / 1184
FRAME 900 / 1184
FRAME 1000 / 1184
FRAME 1100 / 1184


In [41]:
# Stage 3: Temporal filtering... Remove any path that isn't 20 frames or longer

L = deepcopy(newpaths)
temporalcutoff = 20
newpaths = [p for p in L if len(p)> temporalcutoff]        
newpaths = [p for p in newpaths if (p[1][0]-p[0][0] < temporalcutoff)]

stage3tubes = newpaths
putTubesOnVid(newpaths, vidbase+'_outstage3.mp4')

FRAME 100 / 1184
FRAME 200 / 1184
FRAME 300 / 1184
FRAME 400 / 1184
FRAME 500 / 1184
FRAME 600 / 1184
FRAME 700 / 1184
FRAME 800 / 1184
FRAME 900 / 1184
FRAME 1000 / 1184
FRAME 1100 / 1184


In [42]:
# Stage 4: Officially turning paths into TUBES by filling in the temporal blanks

L = deepcopy(newpaths)
tubes = []

#Each slice should be [f, ((X,Y),(b,A),rot), [Vxavg,Vyavg], [R,G,B]]

for i in range(0,len(L)):
    slicestoAdd = []
    for f in range(L[i][0][0], L[i][-1][0]):
        #chunk = [L[i].index(e) for e in L[i] if abs(e[0]-f) < 15]
        
        thisone = [e for e in newpaths[1] if e[0]==f]
        if not thisone: #if we don't have a slice at this frame
            thischunk = []
            a = 2
            while len(thischunk) < 3:
                thischunk = [L[i].index(e) for e in L[i] if abs(e[0]-f)<=a]
                a=a+2
                
            chunkX = [L[i][ind][1][0][0] for ind in thischunk]
            X = sum(chunkX)/len(chunkX)
            chunkY = [L[i][ind][1][0][1] for ind in thischunk]
            Y = sum(chunkY)/len(chunkY)
            chunksmallax = [L[i][ind][1][1][0] for ind in thischunk]
            smallax=max(chunksmallax)
            chunkbigax = [L[i][ind][1][1][1] for ind in thischunk]
            bigax=max(chunkbigax)
            chunkphi = [L[i][ind][1][2] for ind in thischunk]
            phi=max(chunkphi)
            chunkf = [L[i][ind][0] for ind in thischunk]
            Vx = (chunkX[-1] - chunkX[0])/(max(chunkf)+1-min(chunkf))
            Vy = (chunkY[-1] - chunkY[0])/(max(chunkf)+1-min(chunkf))
        
            thisslice = [f, ((X,Y),(smallax,bigax),phi), [Vx,Vy], L[i][0][3]]
            slicestoAdd.append(thisslice)
            
    L[i].extend(slicestoAdd)
            
tubes = L


#Handles when there are 2 ellipses from the same tube on a single frame.
#We choose the bigger one
L = deepcopy(tubes)
tubes = []

for i in range(0,len(L)):
    toRemove = []
    for f in range(1,F):
        #build a list of shit to delete
        A = [e for e in L[i] if e[0] == f]
        if len(A)>1:
            A.sort(key = lambda x: x[1][1][0]**2 + x[1][1][1]**2) 
            toRemove.extend(A[:-1])

    tubes.append([p for p in L[i] if p not in toRemove])
    
#Sorting again... 
for t in tubes:
    t.sort(key = lambda x: x[0])
  

stage4tubes = tubes
putTubesOnVid(tubes, vidbase+'_outstage4.mp4')
putTubesOnVid(tubes, vidbase+'_outstage4nums.mp4', True)

FRAME 100 / 1184
FRAME 200 / 1184
FRAME 300 / 1184
FRAME 400 / 1184
FRAME 500 / 1184
FRAME 600 / 1184
FRAME 700 / 1184
FRAME 800 / 1184
FRAME 900 / 1184
FRAME 1000 / 1184
FRAME 1100 / 1184
FRAME 100 / 1184
FRAME 200 / 1184
FRAME 300 / 1184
FRAME 400 / 1184
FRAME 500 / 1184
FRAME 600 / 1184
FRAME 700 / 1184
FRAME 800 / 1184
FRAME 900 / 1184
FRAME 1000 / 1184
FRAME 1100 / 1184


In [43]:
# Tube merging functions to prepare us for the next stage(s)

def getMerges(L, lookahead):
    mergeList = []
    for i in range(0,len(L)):
        
            
        CVx = [L[i][ind][2][0] for ind in range(-5,0)]
        CVy = [L[i][ind][2][1] for ind in range(-5,0)]
        forVx = sum(CVx)/len(CVx)
        forVy = sum(CVy)/len(CVy)  #ending velocity of tube i
        BVx = [L[i][ind][2][0] for ind in range(0,5)]
        BVy = [L[i][ind][2][1] for ind in range(0,5)]
        backVx = sum(BVx)/len(BVx)
        backVy = sum(BVy)/len(BVy)  #starting velocity of tube i

        
        if len(L[i]) < 10:
            print("BAD")
            continue
        forX = [L[i][ind][1][0][0] for ind in range(-10,0)]
        forY = [L[i][ind][1][0][1] for ind in range(-10,0)]
        backX = [L[i][ind][1][0][0] for ind in range(0,10)]
        backY = [L[i][ind][1][0][1] for ind in range(0,10)]
        
        forVx = (forX[-1] - forX[0]) /10
        forVy = (forY[-1] - forY[0]) /10
        backVx = (backX[-1] - backX[0]) /10
        backVy = (backY[-1] - backY[0]) /10

        predx = (sum(forX)/len(forX)) + (forVx*lookahead)
        predy = (sum(forY)/len(forY)) + (forVy*lookahead)
        predf = L[i][-1][0] + lookahead
        backpredx = (sum(backX)/len(backX)) - (backVx*lookahead)
        backpredy = (sum(backY)/len(backY)) - (backVy*lookahead)
        backpredf = L[i][0][0] - lookahead

        for j in range(0,len(L)):
            if i==j:
                continue

            closeones = [e for e in L[j] if abs(e[0] - predf)<5]
            Bcloseones= [e for e in L[j] if abs(e[0]-backpredf)<5]
            matchFlag = False
            BmatchFlag= False

            for e in closeones:
                r = math.sqrt((e[1][0][0] - predx)**2 + (e[1][0][1] - predy)**2 )
                if (r<100) and (abs(e[2][0] - forVx)<2) and (abs(e[2][1] - forVy)<2):
                    matchFlag = True

            for e in Bcloseones:
                r = math.sqrt((e[1][0][0] - backpredx)**2 + (e[1][0][1] - backpredy)**2 )
                if (r<50) and (abs(e[2][0] - backVx)<2) and (abs(e[2][1] - backVy)<2):
                    BmatchFlag = True

            if (matchFlag):
                mergeList.append([i,j,0])  #j goes in back of i
            if (BmatchFlag):
                mergeList.append([i,j,1])  #j goes in front of i

    nahList = []
    for m in mergeList:
        a = m[0]
        b = m[1]
        
        allVxa = [L[a][ind][2][0] for ind in range(0,len(L[a])) ] 
        Vxa = sum(allVxa)/len(allVxa)
        allVya = [L[a][ind][2][1] for ind in range(0,len(L[a])) ] 
        Vya = sum(allVya)/len(allVya)  

        allVxb = [L[b][ind][2][0] for ind in range(0,len(L[b])) ] 
        Vxb = sum(allVxb)/len(allVxb)
        allVyb = [L[b][ind][2][1] for ind in range(0,len(L[b])) ] 
        Vyb = sum(allVyb)/len(allVyb)

        if (Vxa/Vxb>0) and (Vya/Vyb>0 or (abs(Vya)<1.2 and abs(Vyb)<1.2) ) :
            A = [L[a][ind][0] for ind in range(0,len(L[a]))]
            B = [L[b][ind][0] for ind in range(0,len(L[b]))]
            overlap = [val for val in A if val in B]
            if len(overlap) > min(len(A),len(B))/4:
                nahList.append(m)
        else:
            nahList.append(m)

    mergeList = [m for m in mergeList if m not in nahList]
    
    return mergeList

def doMerge(L, mergeList):
    
    colorList = [t[0][3] for t in L]
    for m in mergeList:
        tlist = []
        for i in range(0,len(L)):
            for e in L[i]:
                if e[3]==colorList[m[0]] or e[3]==colorList[m[1]]:
                    tlist.append(i)

        flattubes = [ell for tube in L for ell in tube]
        cList = [colorList[a] for a in set(tlist)]
        connected = [e for e in flattubes if e[3] in cList]
        for i in set(tlist):
            L[i] = connected
    
    #Remove the extras (duplicates)
    delList = []
    for i in range(0,len(L)):
        if not(L.index(L[i]) == i):
            delList.append(i)
            
    #print(delList)
    Otubes = [L[i] for i in range(0,len(L)) if i not in delList ]
    
    for t in Otubes:
        for e in t:
            e[3] = t[0][3]
    

    return Otubes


In [44]:
# Stage 5: Merging tubes! This first merge is a short term merge (lookahead = 5 frames)

#Sorting again... 
for t in stage4tubes:
    t.sort(key = lambda x: x[0])
    
#Readjusting the velocities based on a sliding window
L = deepcopy(stage4tubes)
Vgap = 10 #calculating velocities across +/- Vgap 
for t in L:
    for i in range(0,len(t)): #for each ellipse
        close = [e for e in t if abs(e[0] - t[i][0])<= Vgap]
        starter = close[0]
        ender = close[-1]
        fgap = ender[0]-starter[0]
        Vx = (ender[1][0][0] - starter[1][0][0]) / (abs(fgap)+1)
        Vy = (ender[1][0][1] - starter[1][0][1]) / (abs(fgap)+1)
        t[i][2] = [Vx, Vy]

lookahead = 5

M5 = getMerges(L,lookahead)
tubes = doMerge(L,M5)
print(M5)

#Gets rid of duplicates in each path (took forever smh)
L= deepcopy(tubes)
tubes = []

for i in range(0,len(L)):
    bigA = []
    for j in range(0,len(L[i])):
        thise = L[i][j]
        A = [L[i].index(e) for e in L[i] if e==thise]
        #print("A:", A)
        bigA.extend(A)
    
    out = [L[i][t] for t in list(set(bigA))]
    tubes.append(out)
    
    
stage5tubes = tubes
putTubesOnVid(tubes, vidbase+'_outstage5.mp4')
putTubesOnVid(tubes, vidbase+'_outstage5nums.mp4', True)

[[7, 12, 0], [7, 52, 0], [16, 17, 0], [16, 56, 0], [25, 34, 0], [34, 25, 1], [52, 7, 1], [54, 56, 0], [55, 17, 0], [55, 56, 0], [56, 55, 1]]
FRAME 100 / 1184
FRAME 200 / 1184
FRAME 300 / 1184
FRAME 400 / 1184
FRAME 500 / 1184
FRAME 600 / 1184
FRAME 700 / 1184
FRAME 800 / 1184
FRAME 900 / 1184
FRAME 1000 / 1184
FRAME 1100 / 1184
FRAME 100 / 1184
FRAME 200 / 1184
FRAME 300 / 1184
FRAME 400 / 1184
FRAME 500 / 1184
FRAME 600 / 1184
FRAME 700 / 1184
FRAME 800 / 1184
FRAME 900 / 1184
FRAME 1000 / 1184
FRAME 1100 / 1184


In [45]:
# Event detection in preparation for the next stage(s)


#Putting into the familiar form. e = [x,y,a,b,phi,f,Vx,Vy,R,G,B, tubenum] 
#Prepping the data for event detection
tubeList=[]
L = deepcopy(tubes)
for tube in L:
    for e in tube:
        ellipse = [e[1][0][0], e[1][0][1], e[1][1][1]/2, e[1][1][0]/2, e[1][2], e[0], e[2][0], e[2][1], e[3][0], e[3][1], e[3][2],L.index(tube) ]
        tubeList.append(ellipse)

DATA = np.array(tubeList)


#This is code that I wrote a while back to detect "events"... when tubes get a little too close for comfort

sThresh = min(2*np.mean(AVGS), 100)
tThresh = 3

eventList = []
#Each event is a "collision" of two tubes t1 and t2 given by [f1,x1,y1,t1,f2,x2,y2,t2]

for f in range(tThresh,F):
    inTime = [ (row[5],row[0],row[1],row[-1]) for row in DATA if abs(row[5] - f) < tThresh ]
    inTimesizes = [(row[2]+row[3],row[-1]) for row in DATA if abs(row[5]-f) < tThresh]
    
    for comprow in inTime:
        notSameTube = np.array([row for row in inTime if row[-1] != comprow[-1]])
        nSTsizes = np.array([row for row in inTimesizes if row[-1] !=comprow[-1]])
        #print(nSTsizes)
        #print()
        
        if notSameTube.size!=0:
            checklocs = [ [ A[1], A[2]] for A in notSameTube] #locations
            thisloc = np.array([comprow[1], comprow[2]])
            #thissize = comprow[3]
            diff = np.linalg.norm((thisloc - checklocs),axis=1)
            sizecheck = min(nSTsizes[:,0])
            #print(diff)
            #print()

            closeIndex = [i for i,x in enumerate(diff) if x < sThresh]
            closeOnes = [notSameTube[i] for i in closeIndex]
            #print(closeOnes)

            for cL in closeOnes:
                CR = list(comprow)
                CR.extend(list(cL))
                eventList.append(CR)
                 

            
majorEventList = []
# Each major event combines the large groups of events into a smaller format [fi, ff, xi, xf, yi, yf, t1, t2]
# where i,f are initial and final and location is an avg of the two tubes

if eventList:
    datubes = np.sort([eventList[0][3], eventList[0][7]])
    startf = int(eventList[0][0])
    startloc = [np.mean((eventList[0][1],eventList[0][5])) , np.mean((eventList[0][2],eventList[0][6]))]

for i in range(len(eventList)):
    E = eventList[i]
    ts=np.sort([E[3], E[7]])
    

    if np.array_equal(ts,datubes):
        continue
    else:
        D=eventList[i-1]
        endf=int(D[0])
        endloc= [np.mean((D[1],D[5])) , np.mean((D[2],D[6]))]
        me = [startf, endf, startloc[0], endloc[0], startloc[1], endloc[1], int(datubes[0]), int(datubes[1])]
        #print(me)
        majorEventList.append(deepcopy(me))
        
        E = eventList[i]
        datubes=np.sort([E[3], E[7]])
        startf = int(E[0])
        startloc = [np.mean((E[1],E[5])) , np.mean((E[2],E[6]))]

Tlist = [ [me[-2],me[-1]] for me in majorEventList]
EventList = []

Tlistset = set(tuple(row) for row in Tlist)
for Tset in Tlistset:
    fList = []
    Tindices = [i for i,x in enumerate(Tlist) if tuple(x) ==Tset]
    for Ti in Tindices:    
        fList.append(majorEventList[Ti][0])
        fList.append(majorEventList[Ti][1])
    fmin = min(fList)
    fmax = max(fList)
    EventList.append([fmin, fmax, majorEventList[Tindices[0]][2], majorEventList[Tindices[-1]][2], majorEventList[Tindices[0]][4], majorEventList[Tindices[-1]][4], Tset[0], Tset[1] ])
    
        
eventlist = EventList
eventList = EventList
print(len(EventList))
for e in EventList:
    print(e)
        
L = deepcopy(tubes)
deltaList = []

for t in L:
    dxy= []
    dV = []
    dab= []
    for i in range(1,len(t)):
        this = t[i]
        old = t[i-1]
        dxy.append(math.sqrt((this[1][0][0]-old[1][0][0])**2 + (this[1][0][1]-old[1][0][1])**2))
        dV.append(math.sqrt((this[2][0]-old[2][0])**2 + (this[2][1]-old[2][1])**2))
        dab.append(math.sqrt((this[1][1][0]-old[1][1][0])**2 + (this[1][1][1]-old[1][1][1])**2))
    deltaList.append([dxy, dV, dab])


86
[629, 649, 688.7489624023438, 687.1180725097656, 12.641826152801514, 26.46265983581543, 54, 55]
[635, 653, 289.56365966796875, 284.6998291015625, 9.921762704849243, 4.537713050842285, 31, 57]
[10, 303, 518.1297149658203, 492.59619140625, 21.396230697631836, 134.97190856933594, 11, 47]
[10, 299, 498.2071990966797, 118.78411865234375, 8.967913627624512, 142.6163787841797, 11, 48]
[644, 650, 790.4483032226562, 785.9803466796875, 35.80881690979004, 37.720733642578125, 27, 55]
[893, 923, 911.3034362792969, 832.1015014648438, 295.7783660888672, 299.25685119628906, 32, 64]
[646, 653, 916.4645690917969, 911.4422912597656, 310.15513610839844, 307.0495147705078, 21, 53]
[551, 702, 999.003173828125, 970.5670166015625, 236.86614227294922, 259.55921936035156, 52, 53]
[753, 809, 910.2206420898438, 855.8600769042969, 260.53707122802734, 287.9223937988281, 32, 51]
[10, 77, 1010.2188720703125, 1002.5150756835938, 222.74927520751953, 239.66647338867188, 4, 46]
[1166, 1181, 364.28981018066406, 346.976

In [46]:
# Stage 5.5: Makes sure that tubes are connected across the merge without gaps.

for t in tubes:
    t.sort(key = lambda x: x[0])

L = deepcopy(tubes)    
    
for t in L:
    if (len(t) >= (t[-1][0] - t[0][0] + 1)):
        continue
        
    else:
        print("Gaps in tube", tubes.index(t))
        toAdd = []
        have = [e[0] for e in t]
        donthave = [num for num in range(t[0][0], t[-1][0]) if num not in have]
        dlist = []
        for k, g in groupby(enumerate(donthave), lambda ix :ix[0]-ix[1]):
            dlist.append(list(map(itemgetter(1), g) ))
        for gap in dlist:
            preve = [e for e in t if e[0]==gap[0]-1][0]
            prevsize = (preve[1][1][0]+preve[1][1][1])/2
            nextegroup = [e for e in t if (e[0]>= gap[-1]) and (e[0]<=gap[-1]+10) ]
            nextegroup.sort(key = lambda x: (x[1][1][0] + x[1][1][1])/2 - prevsize)
            nexte = nextegroup[0]
            fgap = len(gap)+1
            for f in gap:
                framediff = f - (gap[0]-1)
                
                X = (nexte[1][0][0] - preve[1][0][0]) * (framediff/fgap) + preve[1][0][0]
                Y = (nexte[1][0][1] - preve[1][0][1]) * (framediff/fgap) + preve[1][0][1]
                smallax = max(nexte[1][1][0],preve[1][1][0]) 
                bigax = max(nexte[1][1][1],preve[1][1][1]) 
                phi = preve[1][2]
                #phi = (nexte[1][2] - preve[1][2]) * (framediff/fgap) + preve[1][2]
                Vx = (X - preve[1][0][0])/framediff
                Vy = (Y - preve[1][0][1])/framediff
                RGB = preve[3]
                
                newe = [f, ((X,Y), (smallax,bigax), phi), [Vx,Vy], RGB]
                toAdd.append(newe)
          
        t.extend(toAdd)  
            
tubes = L
for t in tubes:
    t.sort(key = lambda x: x[0])
    
stage55tubes = tubes    


Gaps in tube 3
Gaps in tube 4
Gaps in tube 10
Gaps in tube 12
Gaps in tube 14
Gaps in tube 16
Gaps in tube 17
Gaps in tube 18
Gaps in tube 19
Gaps in tube 20
Gaps in tube 21
Gaps in tube 22
Gaps in tube 23
Gaps in tube 24
Gaps in tube 25
Gaps in tube 26
Gaps in tube 27
Gaps in tube 28
Gaps in tube 29
Gaps in tube 31
Gaps in tube 33
Gaps in tube 34
Gaps in tube 35
Gaps in tube 36
Gaps in tube 37
Gaps in tube 40
Gaps in tube 42
Gaps in tube 43
Gaps in tube 44
Gaps in tube 45
Gaps in tube 46
Gaps in tube 47
Gaps in tube 48
Gaps in tube 49
Gaps in tube 50
Gaps in tube 51
Gaps in tube 52
Gaps in tube 53
Gaps in tube 56
Gaps in tube 57
Gaps in tube 58
Gaps in tube 59
Gaps in tube 60
Gaps in tube 61
Gaps in tube 62
Gaps in tube 63
Gaps in tube 64
Gaps in tube 65
Gaps in tube 66
Gaps in tube 67


In [47]:
# Stage 6: Detects and handles the collision where one tube "swallows" another by 
# implementing short term trackers on each (pre-collision) before reconnecting whenever possible.
# Used to be known as stage 8, but got moved up here in the process.

    

L = deepcopy(stage55tubes)
tracks = []
closeThresh = 30
steadyThresh = 10
hitList = []

for i in range(0,len(L)):
    if L[i]:
        lastf = L[i][-1][0]
        #print("Lastf", lastf)
        a = [e[-2] for e in EventList if i in e[-2:]]
        b = [e[-1] for e in EventList if i in e[-2:]]
        etubes = list(set(a+b))
        if etubes:
            etubes.remove(i)   #now we have a list of tubes to look at
            for t in etubes:
                #Find the start of this event...
                thisEvent = [e for e in EventList if (i in e[-2:]) and (t in e[-2:])][0]
                #print(thisEvent)
                eventStart = thisEvent[0]
                eventEnd = thisEvent[1]
                startf = L[t][0][0]
                dpos = np.mean(deltaList[t][0][lastf-startf-3:lastf-startf+3])
                dsize= np.mean(deltaList[t][2][lastf-startf-3:lastf-startf+3])
                if math.isnan(dpos) or math.isnan(dsize):
                    continue
                if (np.mean(deltaList[t][0]) < dpos) and (np.mean(deltaList[t][2]) < dsize) and (abs(eventStart-lastf)<=20 or abs(eventEnd-lastf)<=20):
                    print("We got a hit!")
                    print("First tube is ", i, "and swallowing tube is ", t)
                    print("Average position delta:", np.mean(deltaList[t][0]) )
                    print("Here it's: ",dpos)
                    print("Average size delta:", np.mean(deltaList[t][2]) )
                    print("Here it's ",dsize)
                    hitList.append(i)

                    track1 = []
                    track2 = []
                    efirst = [e for e in L[i] if (e[0]>=lastf-3) and (e[0] <=lastf)]
                    esec = [e for e in L[t] if (e[0]>=lastf-3) and (e[0] <=lastf)]
                    if not(efirst) or not(esec):
                        continue
                    efirst = efirst[0]
                    esec = esec[0]
                    tracker1 = cv2.TrackerCSRT_create()
                    tracker2 = cv2.TrackerCSRT_create()
                    bbox1 = getBoxFromEllipse(efirst)
                    bbox2 = getBoxFromEllipse(esec)
                    ok1 = tracker1.init(invid[int(lastf-3)], bbox1)
                    ok2 = tracker2.init(invid[int(lastf-3)], bbox2)
                    track1.append((int(lastf-3), bbox1))
                    track2.append((int(lastf-3), bbox2))
                    L[t] = [e for e in L[t] if e[0] <= lastf-3]

                    OutofBoundsFlags = [False, False]
                    NewTubeFlags = [False, False]
                    SteadyTrackFlags = [False, False]
                    f = lastf-3

                    while not(OutofBoundsFlags[0]) and not(NewTubeFlags[0]) and (f<F-1) :
                        f = f+1
                        r1= 9999
                        ok1, bbox1 = tracker1.update(invid[int(f)])
                        center1 = (bbox1[0]+bbox1[2]/2 , bbox1[1]+bbox1[3]/2)

                        if ok1:
                            track1.append((int(f), bbox1))
                            

                        #Check for NEW tube super close to the bbox (after about 20 frames)
                        if f > lastf+30:
                            for thistube in L:  
                                if L.index(thistube)==t or L.index(thistube)==i: #not including the collision one
                                    continue
                                Thise = [e for e in thistube if e[0] == f]
                                if Thise:
                                    thise = Thise[0]
                                    r1 = math.sqrt((thise[1][0][0] - center1[0])**2 + (thise[1][0][1] - center1[1])**2 ) 
                                if r1 < closeThresh:
                                    print("Tube ", L.index(thistube),"intersects at", thise)
                                    NewTubeFlags[0] = True
                                    newtubef = f
                                    print("Tube", L.index(thistube),"was",r1, "away from tube",i," at frame",newtubef)
                                    break

                        #Check for OOB        
                        if (center1[0] > W-30) or (bbox1[0]<30) or (center1[1]>H-5) or (bbox1[1]<0):
                            OutofBoundsFlags[0] = True
                            print("Went outta bounds.")
                        '''  
                        #Check for a super steady track.
                        if len(track1) > steadyThresh:
                            ssum = 0
                            for k in range(-steadyThresh,-1):
                                rt = math.sqrt((track1[k][1][0] - track1[k+1][1][0])**2 + (track1[k][1][1] - track1[k+1][1][1])**2)
                                ssum = ssum + rt
                            if ssum<steadyThresh:
                                SteadyTrackFlags[0] = True
                                print("Track is too steady. Probably done.")
                                print("Movement in the last 30 frames = ", ssum,"pixels")
                        '''
                    if len(track1) >= 25:
                        #See if the last thing on the track is similar to something about 30 frames ago...
                        #If so, delete any track box within a 10 pixel radiuss of that one
                        rt = math.sqrt((track1[-1][1][0] - track1[-20][1][0])**2 + (track1[-1][1][1] - track1[-20][1][1])**2)
                        if rt<2:
                            print("At least part of the track on", i, "was too steady to trust.")
                            savetr = track1[0]
                            steadyList = [tr for tr in track1 if math.sqrt((track1[-1][1][0] - tr[1][0])**2 + (track1[-1][1][1] - tr[1][1])**2) < 30]
                            track1 = [tr for tr in track1 if tr not in steadyList]
                            if not(track1):
                                track1 = [savetr]

                        
                    #Go thru track1 and turn boxes into ellipses and put them in the proper tube
                    toAdd = []
                    for tr in track1:
                        ell = [tr[0], getEllipseFromBox(tr[1], efirst[1][2]), efirst[2], efirst[3]]
                        toAdd.append(ell)
                    L[i].extend(toAdd)
                    #If NewTubeFlag, then everything in the new tube
                    if NewTubeFlags[0] and (newtubef<=track1[-1][0]):
                        L[i] = [e for e in L[i] if e[0] < newtubef] 
                        toAdd = [e for e in thistube if e[0]>=newtubef]
                        for ell in toAdd:
                            ell[3] = L[i][0][3]
                        L[i].extend(toAdd)



                    f = lastf-3
                    while not(OutofBoundsFlags[1]) and not(NewTubeFlags[1]) and (f<F-1):
                        f = f+1
                        r2 =9999
                        ok2, bbox2 = tracker2.update(invid[int(f)])
                        center2 = (bbox2[0]+bbox2[2]/2 , bbox2[1]+bbox2[3]/2)

                        if ok2:
                            track2.append((int(f), bbox2))

                        #Check for NEW tube super close to the bbox
                        if f > lastf+30:
                            for thistube in L:
                                if L.index(thistube)==t or L.index(thistube)==i: #not including the collision one
                                    continue
                                Thise = [e for e in thistube if e[0] == f]
                                if Thise:
                                    thise = Thise[0]
                                    r2 = math.sqrt((thise[1][0][0] - center2[0])**2 + (thise[1][0][1] - center2[1])**2 ) 
                                if r2 < closeThresh:
                                    print("Tube ", L.index(thistube),"intersects at", thise)
                                    NewTubeFlags[1] = True
                                    newtubef = f
                                    print("Tube", L.index(thistube),"was",r2,  "away from tube",t," at frame",newtubef)
                                    
                                    break

                        #Check for OOB
                        if (center2[0] > W-30) or (bbox2[0]<30) or (center2[1]>H-5) or (bbox2[1]<0):
                            OutofBoundsFlags[1] = True
                        '''
                        #Check for a super steady track.
                        if len(track2) > steadyThresh:
                            ssum = 0
                            for k in range(-steadyThresh,-1):
                                rt = math.sqrt((track2[k][1][0] - track2[k+1][1][0])**2 + (track2[k][1][1] - track2[k+1][1][1])**2)
                                ssum = ssum + rt
                            if ssum<steadyThresh:
                                SteadyTrackFlags[1] = True
                                print("Track is too steady. Probably done.")
                                print("Movement in the last 30 frames = ", ssum,"pixels")
                        ''' 
                    if len(track2)>=25:       
                        #See if the last thing on the track is similar to something about 30 frames ago...
                        #If so, delete any track box within a 10 pixel radiuss of that one
                        rt = math.sqrt((track2[-1][1][0] - track2[-20][1][0])**2 + (track2[-1][1][1] - track2[-20][1][1])**2)
                        if rt<2:
                            print("At least part of the track on", t, "was too steady to trust.")
                            savetr = track2[0]
                            steadyList = [tr for tr in track2 if math.sqrt((track2[-1][1][0] - tr[1][0])**2 + (track2[-1][1][1] - tr[1][1])**2) < 30]
                            track2 = [tr for tr in track2 if tr not in steadyList]
                            if not(track2):
                                track2 = [savetr]

                    #Go thru track2 and turn boxes into ellipses and put them in the proper tube
                    toAdd = []
                    #print(track2)
                    for tr in track2:
                        ell = [tr[0], getEllipseFromBox(tr[1], esec[1][2]), esec[2], esec[3]]
                        toAdd.append(ell)
                    L[t].extend(toAdd)
                    #If NewTubeFlag, then everything in the new tube
                    if NewTubeFlags[1] and (newtubef<=track2[-1][0]):
                        print("THISTUBE" , L.index(thistube))
                        L[t] = [e for e in L[t] if e[0] < newtubef] 
                        toAdd = [e for e in thistube if e[0]>=newtubef]
                        for ell in toAdd:
                            ell[3] = L[t][0][3]
                        L[t].extend(toAdd)



                    tracks.append(track1)
                    tracks.append(track2)

L = [tube for tube in L if tube!=[]]
tubes = L

#Readjusting the velocities based on a sliding window
L = deepcopy(tubes)
Vgap = 3  #calculating velocities across +/- Vgap 
for t in L:
    for i in range(0,len(t)): #for each ellipse
        close = [e for e in t if abs(e[0] - t[i][0])<= Vgap]
        starter = close[0]
        ender = close[-1]
        fgap = ender[0]-starter[0]
        Vx = (ender[1][0][0] - starter[1][0][0]) / (abs(fgap)+1)
        Vy = (ender[1][0][1] - starter[1][0][1]) / (abs(fgap)+1)
        t[i][2] = [Vx, Vy]

#if the velocity puts you OOB soon, you out.
for h in hitList:
    startf = L[h][0][0]
    for e in L[h]:
        predX = e[1][0][0] + e[1][1][0]*3
        predY = e[1][0][1] + e[1][1][1]*3
        if (predX > W or predX<0) and(e[0]-startf >30):
            print(h,"is messed up horiz at frame", e[0])
            #L[h] = [ell for ell in L[h] if ell[0]<=e[0]]
            break

tubes = L

print("Done.")

stage6tubes = tubes
#putTracksOnVid(tracks,vidbase+'_outstage5nums.mp4',vidbase+'_outstage6tracksUGLY.mp4')
#putTubesOnVid(tubes, vidbase+'_outstage6UGLY.mp4')
putTubesOnVid(tubes, vidbase+'_outstage6nums.mp4', True)

C:\Users\khari\Anaconda3\envs\py36\lib\site-packages\numpy\core\fromnumeric.py:3335: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
C:\Users\khari\Anaconda3\envs\py36\lib\site-packages\numpy\core\_methods.py:161: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


We got a hit!
First tube is  1 and swallowing tube is  39
Average position delta: 11.48871945255591
Here it's:  172.35497996804114
Average size delta: 8.184600647937815
Here it's  50.278153537073855
We got a hit!
First tube is  2 and swallowing tube is  3
Average position delta: 19.097016830521003
Here it's:  38.66849046949064
Average size delta: 8.273770901239187
Here it's  15.265285075175578
Tube  15 intersects at [340, ((429.7845764160156, 481.1812438964844), (13.063085556030273, 15.42496395111084), 69.64412689208984), [4.393056233723958, -0.3248341878255208], [133, 200, 2]]
Tube 15 was 0.3040025865432421 away from tube 2  at frame 340
At least part of the track on 2 was too steady to trust.
Tube  42 intersects at [59, ((380.9623260498047, 484.728759765625), (17.712356567382812, 18.32282257080078), 107.60762023925781), [-0.4138336181640625, 0.054718017578125], [165, 170, 182]]
Tube 42 was 5.182254137122642 away from tube 3  at frame 59
At least part of the track on 3 was too steady 

Went outta bounds.
At least part of the track on 29 was too steady to trust.
We got a hit!
First tube is  57 and swallowing tube is  56
Average position delta: 32.24502240800396
Here it's:  44.538414718707855
Average size delta: 17.80240842417319
Here it's  25.71932645255403
Went outta bounds.
Tube  31 intersects at [684, ((152.95945358276367, 36.71322250366211), (35.294090270996094, 70.58818054199219), 80.80762481689453), [-4.762107849121094, -0.1467304229736328], [186, 190, 84]]
Tube 31 was 0.0 away from tube 56  at frame 684
At least part of the track on 56 was too steady to trust.
We got a hit!
First tube is  60 and swallowing tube is  32
Average position delta: 3.1785203555571844
Here it's:  23.725867347497157
Average size delta: 6.223219074637141
Here it's  53.8249164120216
We got a hit!
First tube is  62 and swallowing tube is  33
Average position delta: 6.054321977479282
Here it's:  11.642363257735218
Average size delta: 8.417728703638732
Here it's  15.706938578209098
At least 

In [48]:
writeTubelist(vidbase+'_s6Tubelist.csv', stage6tubes)
#tubes = readTubelist(vidbase+'_s6Tubelist.csv')
#tubes = readTubelist('int6_s8Tubelist.csv')


In [27]:
# Stage 6.5: Second tube merging. This one is medium-term (look ahead = 50 frames)


L = deepcopy(stage6tubes)
lookahead = 50

M60 = getMerges(L,lookahead)
newtubes = doMerge(L,M60)
print(M60)

#Gets rid of duplicates in each path (took forever smh)
L= deepcopy(newtubes)
tubes = []

for i in range(0,len(L)):
    bigA = []
    for j in range(0,len(L[i])):
        thise = L[i][j]
        A = [L[i].index(e) for e in L[i] if e==thise]
        #print("A:", A)
        bigA.extend(A)
    
    out = [L[i][t] for t in list(set(bigA))]
    tubes.append(out)
    
stage7tubes = tubes
putTubesOnVid(tubes, vidbase+'_outstage6.5.mp4')
putTubesOnVid(tubes, vidbase+'_outstage6.5nums.mp4', True)

[[10, 33, 0], [16, 30, 0], [28, 30, 0]]
FRAME 100 / 1375
FRAME 200 / 1375
FRAME 300 / 1375
FRAME 400 / 1375
FRAME 500 / 1375
FRAME 600 / 1375
FRAME 700 / 1375
FRAME 800 / 1375
FRAME 900 / 1375
FRAME 1000 / 1375
FRAME 1100 / 1375
FRAME 1200 / 1375
FRAME 1300 / 1375
FRAME 100 / 1375
FRAME 200 / 1375
FRAME 300 / 1375
FRAME 400 / 1375
FRAME 500 / 1375
FRAME 600 / 1375
FRAME 700 / 1375
FRAME 800 / 1375
FRAME 900 / 1375
FRAME 1000 / 1375
FRAME 1100 / 1375
FRAME 1200 / 1375
FRAME 1300 / 1375


In [28]:
# Stage 7: Second tube merging. This one is long-term (look ahead = 60 frames)

L = deepcopy(tubes)
lookahead = 60

M60 = getMerges(L,lookahead)
newtubes = doMerge(L,M60)
print(M60)



#Gets rid of duplicates in each path (took forever smh)
L= deepcopy(newtubes)
tubes = []

for i in range(0,len(L)):
    bigA = []
    for j in range(0,len(L[i])):
        thise = L[i][j]
        A = [L[i].index(e) for e in L[i] if e==thise]
        #print("A:", A)
        bigA.extend(A)
    
    out = [L[i][t] for t in list(set(bigA))]
    tubes.append(out)


stage7tubes = tubes
putTubesOnVid(tubes, vidbase+'_outstage7.mp4')
putTubesOnVid(tubes, vidbase+'_outstage7nums.mp4', True)

[]
FRAME 100 / 1375
FRAME 200 / 1375
FRAME 300 / 1375
FRAME 400 / 1375
FRAME 500 / 1375
FRAME 600 / 1375
FRAME 700 / 1375
FRAME 800 / 1375
FRAME 900 / 1375
FRAME 1000 / 1375
FRAME 1100 / 1375
FRAME 1200 / 1375
FRAME 1300 / 1375
FRAME 100 / 1375
FRAME 200 / 1375
FRAME 300 / 1375
FRAME 400 / 1375
FRAME 500 / 1375
FRAME 600 / 1375
FRAME 700 / 1375
FRAME 800 / 1375
FRAME 900 / 1375
FRAME 1000 / 1375
FRAME 1100 / 1375
FRAME 1200 / 1375
FRAME 1300 / 1375


In [29]:
# Stage 8: Makes sure that tubes are connected across the merge without gaps.

L = deepcopy(tubes)
for i in range(0,len(L)):
    for f in range(1,F):
        A = [e for e in L[i] if e[0] == f] #list of ellipses on this frame
        if len(A)>1:
            A.sort(key = lambda x: x[1][1][0]**2 + x[1][1][1]**2)
            L[i] = [e for e in L[i] if e not in A[:-1]] # get rid of the smaller ones           
tubes = L

for t in tubes:
    t.sort(key = lambda x: x[0])

L = deepcopy(tubes)    
    
for t in L:
    if (len(t) >= (t[-1][0] - t[0][0] + 1)):
        continue
        
    else:
        print("Gaps in tube", tubes.index(t))
        toAdd = []
        have = [e[0] for e in t]
        donthave = [num for num in range(t[0][0], t[-1][0]) if num not in have]
        dlist = []
        for k, g in groupby(enumerate(donthave), lambda ix :ix[0]-ix[1]):
            dlist.append(list(map(itemgetter(1), g) ))
        for gap in dlist:
            preve = [e for e in t if e[0]==gap[0]-1][0]
            prevsize = (preve[1][1][0]+preve[1][1][1])/2
            nextegroup = [e for e in t if (e[0]>= gap[-1]) and (e[0]<=gap[-1]+10) ]
            nextegroup.sort(key = lambda x: (x[1][1][0] + x[1][1][1])/2 - prevsize)
            nexte = nextegroup[0]
            fgap = len(gap)+1
            for f in gap:
                framediff = f - (gap[0]-1)
                
                X = (nexte[1][0][0] - preve[1][0][0]) * (framediff/fgap) + preve[1][0][0]
                Y = (nexte[1][0][1] - preve[1][0][1]) * (framediff/fgap) + preve[1][0][1]
                smallax = max(nexte[1][1][0],preve[1][1][0]) 
                bigax = max(nexte[1][1][1],preve[1][1][1]) 
                phi = preve[1][2]
                #phi = (nexte[1][2] - preve[1][2]) * (framediff/fgap) + preve[1][2]
                Vx = (X - preve[1][0][0])/framediff
                Vy = (Y - preve[1][0][1])/framediff
                RGB = preve[3]
                
                newe = [f, ((X,Y), (smallax,bigax), phi), [Vx,Vy], RGB]
                toAdd.append(newe)
          
        t.extend(toAdd)  
            
for t in tubes:
    t.sort(key = lambda x: x[0])
    
    


tubes = L

stage8tubes = tubes    
putTubesOnVid(tubes, vidbase+'_outstage8.mp4')
putTubesOnVid(tubes, vidbase+'_outstage8numsFORSHOW.mp4', True)

Gaps in tube 10
FRAME 100 / 1375
FRAME 200 / 1375
FRAME 300 / 1375
FRAME 400 / 1375
FRAME 500 / 1375
FRAME 600 / 1375
FRAME 700 / 1375
FRAME 800 / 1375
FRAME 900 / 1375
FRAME 1000 / 1375
FRAME 1100 / 1375
FRAME 1200 / 1375
FRAME 1300 / 1375
FRAME 100 / 1375
FRAME 200 / 1375
FRAME 300 / 1375
FRAME 400 / 1375
FRAME 500 / 1375
FRAME 600 / 1375
FRAME 700 / 1375
FRAME 800 / 1375
FRAME 900 / 1375
FRAME 1000 / 1375
FRAME 1100 / 1375
FRAME 1200 / 1375
FRAME 1300 / 1375


In [30]:
writeTubelist(vidbase+'_s8Tubelist.csv', tubes)
#tubes = readTubelist(vidbase+'_s8Tubelist.csv')
#tubes = readTubelist('int6_s8Tubelist.csv')


In [ ]:
%matplotlib notebook

# Plotting tubes

def plotTubes(tubes, which = []):
    fig = plt.figure()
    ax = plt.axes(projection='3d')
    colors = ['r','g','b','c','m','y','k','xkcd:hot pink','xkcd:sand','xkcd:mint','xkcd:wine red']
    ax.set_xlabel('Time')
    ax.set_ylabel('X')
    ax.set_zlabel('Y')
    
    if not(which):
        for t in tubes:
            Ts = [e[0] for e in t]
            Xs = [e[1][0][0] for e in t]
            Ys = [e[1][0][1] for e in t]
            #Cs = t[0][3]
            ax.scatter(Ts,Xs,Ys,c=random.choice(colors), marker='.')

    else:
        for w in set(which):
            tubeToPlot = tubes[w]
            Ts = [e[0] for e in tubeToPlot]
            Xs = [e[1][0][0] for e in tubeToPlot]
            Ys = [e[1][0][1] for e in tubeToPlot]
            #Cs = [e[3] for e in tubeToPlot]
            ax.scatter(Ts,Xs,Ys,c=colors[which.index(w)], marker='.')
            
    plt.show()
    
def plotSubTubes(tubes,labelList, which = []):
    fig = plt.figure()
    ax = plt.axes(projection='3d')
    colors = ['r','g','b','c','m','y','k','xkcd:hot pink','xkcd:sand','xkcd:mint','xkcd:wine red']
    ax.set_xlabel('Time')
    ax.set_ylabel('X')
    ax.set_zlabel('Y')
    
    LL = []
    #Not plotting outliers for now
    for l in labelList:
        l2 = [x+1 if x==-1 else x for x in l]
        LL.append(l2)
        
    
    if not(which):
        for t in tubes:
            for e in t:
                T = e[0]
                X = e[1][0][0]
                Y = e[1][0][1] 
                L = LL[tubes.index(t)][t.index(e)]
                ax.scatter(T,X,Y,c=colors[L], marker='.')

    else:
        for w in set(which):
            tubeToPlot = tubes[w]
            for e in tubeToPlot:
                T = e[0]
                X = e[1][0][0] 
                Y = e[1][0][1] 
                L = LL[w][tubeToPlot.index(e)]
                #Cs = [e[3] for e in tubeToPlot]
                ax.scatter(T,X,Y,c=colors[L], marker='.')
            
    plt.show()
        

#plotSubTubes(tubes,which=[0,1,2])

In [ ]:
print(labelList[0])

In [ ]:
#Quick DBSCAN Plotting
L=deepcopy(tubes)

labelList = []
for t in L:
    XforDB = np.array([[e[0],e[1][0][0], e[1][0][1]] for e in t])
    DBout = DBSCAN(eps=0.6*np.mean(AVGS), min_samples = 10).fit(XforDB)
    labelList.append(DBout.labels_)
    
save1 = labelList[0]
save2 = labelList[1]
save3 = labelList[2]
plotSubTubes(tubes,labelList,which=[0])

In [ ]:
#SAVE (AND LOAD) THE TUBES VARIABLE IN A TUBELIST (.TXT) SO I DON'T HAVTA KEEP RUNNING THE DAMN CODE


           

    
#writeTubelist(vidbase+'_s8Tubelist.csv', tubes)
tubes = readTubelist(vidbase+'_s8Tubelist.csv')
#tubes = readTubelist('int6_s8Tubelist.csv')



In [31]:
#STAGE 9---------this block is kinda long

# Using DBSCAN on each tube 

L=deepcopy(tubes)



labelList = []
for t in L:
    XforDB = np.array([[e[0],e[1][0][0], e[1][0][1]] for e in t])
    DBout = DBSCAN(eps=0.6*np.mean(AVGS), min_samples = 10).fit(XforDB)
    labelList.append(DBout.labels_)
    
#Populated labelList... now FIND OUT WHICH HAVE SUBTUBES AND PLOT THEM
#Thinking len(set()) > 2... but then the -1 thing smh idk
haveSubs = []
haveGlides=[]
haveSteadys=[]
for t in range(0,len(labelList)):
    labs = set(labelList[t])
    if -1 in labs:
        labs.remove(-1)
    if len(labs)>=2:
        haveSubs.append(t)
  

print(haveSubs)


#Readjusting the velocities based on a sliding window
L = deepcopy(tubes)
Vgap = 3  #calculating velocities across +/- Vgap 
for t in L:
    for i in range(0,len(t)): #for each ellipse
        close = [e for e in t if abs(e[0] - t[i][0])<= Vgap]
        starter = close[0]
        ender = close[-1]
        fgap = ender[0]-starter[0]
        Vx = (ender[1][0][0] - starter[1][0][0]) / (abs(fgap)+1)
        Vy = (ender[1][0][1] - starter[1][0][1]) / (abs(fgap)+1)
        t[i][2] = [Vx, Vy]


# Recognize glides and steadys within a tube (and maybe split)
tubestoAdd=[]
for i in range(0,len(L)):
    scount=0
    gcount=0
    j=0
    slist=[]
    glist=[]
    while j+1 < len(L[i]):
        e= L[i][j]
        startf=e[0]
        
        if min([abs(a) for a in e[2]]) < 0.2 and max([abs(a) for a in e[2]])<0.5 :
            scount=scount+1
        else:
            scount=0
        
        if abs(e[2][0]-L[i][j-1][2][0])<0.1 and abs(e[2][1]-L[i][j-1][2][1])<0.1 :
            gcount=gcount+1
        else:
            gcount=0
            
        if scount>30:
            slist.append(startf)
        if gcount>30:
            #print("Gliding at tube", i)
            glist.append(startf)   
        j=j+1
        
    #print("Tube", i)
    #print(slist)
    #print(glist)
    if (len(slist)>3) and (len(glist)<len(slist)/2): #steady and not gliding...
        # Make a split. Add to haveSubs. Change labelList
        A = [e for e in L[i] if (e[0]<min(slist)-30)]
        B = [e for e in L[i] if (e[0]>min(slist)-30) and (e[0]<=max(slist))]
        C = [e for e in L[i] if (e[0]>max(slist))]
        haveSteadys.append(i)
        Ai = [L[i].index(e) for e in A]
        Bi = [L[i].index(e) for e in B]
        Ci = [L[i].index(e) for e in C]
        for a in Ai:
            labelList[i][a]=0
        for b in Bi:
            labelList[i][b]=1
        for c in Ci:
            labelList[i][c]=2
    
    if (len(glist)>3) and (len(slist)<len(glist)/2): #gliding, not steady...
        # Make a split. Add to haveSubs. Change labelList
        A = [e for e in L[i] if (e[0]<min(glist)-30)]
        B = [e for e in L[i] if (e[0]>min(glist)-30) and (e[0]<=max(glist))]
        C = [e for e in L[i] if (e[0]>max(glist))]
        haveGlides.append(i)
        Ai = [L[i].index(e) for e in A]
        Bi = [L[i].index(e) for e in B]
        Ci = [L[i].index(e) for e in C]
        #L[i] = [e for e in L[i] if (e in A) or (e in C)] #no glide
        #for a in Ai:
        #    labelList[i][a]=0
        #for b in Bi:
        #    labelList[i][b]=2
        #for c in Ci:
        #    labelList[i][c]=1
        #labelList[i] = [labelList[i][ind] for ind in range(0,len(labelList[i])) if (ind in Ai) or (ind in Ci) ]

print()
print("Has subtubes:", haveSubs)
print("Has glides:" ,haveGlides)
print("Has steadys:", haveSteadys)
    
    
#Want to remove any subtube (and anything after it) that is steady
#Only considering things with subtubes (SECOND TRY)

hasStuff = list(set(haveSteadys+haveSubs))

for h in hasStuff:
    STlabels = labelList[h]
    T = L[h]
    for l in set(STlabels): #for each subtube
        ST = [T[ind] for ind in range(0,len(STlabels)) if STlabels[ind]==l] #actual subtube
        steadycount=0
        count=0
        for e in ST:
            count=count+1
            if abs(e[2][0])<0.1 and abs(e[2][1])<0.1:
                steadycount=steadycount+1
        perc = steadycount/count
        print(perc, "on tube",h," subtube", l)
        
        
        if perc > 0.5 and l!=-1:
            firstind = list(STlabels).index(l)
            print("Percent steady=",perc,"on tube",h," subtube", l, "starting at", firstind)
            #labelList[h] = [STlabels[ind] for ind in range(0,len(STlabels)) if STlabels[ind]!=l]
            #DBtubes[h] = [T[ind] for ind in range(0,len(T)) if STlabels[ind]!=l]
            labelList[h] = STlabels[0:firstind]
            L[h] = T[0:firstind]
            break
                        

#print(len(labelList))
#print(len(L))
#L = deepcopy(DBtubes)
            
#Remove empty tubes... clear them from haveSubs and labelList also
toDel = [i for i in range(0,len(L)) if len(L[i])==0]
toDel.sort(reverse=True)
print("Whole tubes to delete:", toDel)
for d in toDel:
    haveSubs = [h-1  if h>d else h for h in haveSubs]
    haveSubs.remove(d)
    haveGlides = [h-1 if h>d else h for h in haveGlides]
    if d in haveGlides:
        haveGlides.remove(d)
    
DBtubes = [t for t in L if L.index(t) not in toDel]
labelList = [labelList[i] for i in range(0,len(labelList)) if i not in toDel]

print(haveSubs)
#putSomeTubesOnVid(L, vidbase+'_outstage9numsNoSteadys.mp4',which = haveSubs )


#DBtubes = deepcopy(L)
for t in range(0,len(DBtubes)):
    RGB0 = DBtubes[t][0][3]
    RGB1 = [random.randint(0,255), random.randint(0,255), random.randint(0,255)]
    RGB2 = [random.randint(0,255), random.randint(0,255), random.randint(0,255)]
    RGB3 = [random.randint(0,255), random.randint(0,255), random.randint(0,255)]
    RGB4 = [random.randint(0,255), random.randint(0,255), random.randint(0,255)]
    RGB5 = [random.randint(0,255), random.randint(0,255), random.randint(0,255)]
    RGB6 = [random.randint(0,255), random.randint(0,255), random.randint(0,255)]
    RGB7 = [random.randint(0,255), random.randint(0,255), random.randint(0,255)]
    RGB8 = [random.randint(0,255), random.randint(0,255), random.randint(0,255)]
    RGBM1 = [0, 0, 255]
    
    for i in range(0,len(DBtubes[t])):
        if labelList[t][i]==0:
            DBtubes[t][i][3] = RGB0
        elif labelList[t][i]==1:
            DBtubes[t][i][3] = RGB1
        elif labelList[t][i]==2:
            DBtubes[t][i][3] = RGB2
        elif labelList[t][i]==3:
            DBtubes[t][i][3] = RGB3
        elif labelList[t][i]==4:
            DBtubes[t][i][3] = RGB4
        elif labelList[t][i]==5:
            DBtubes[t][i][3] = RGB5
        elif labelList[t][i]==6:
            DBtubes[t][i][3] = RGB6
        elif labelList[t][i]==7:
            DBtubes[t][i][3] = RGB7
        elif labelList[t][i]==8:
            DBtubes[t][i][3] = RGB8
        else:
            DBtubes[t][i][3] = RGBM1
    
    
#putSomeTubesOnVid(DBtubes, vidbase+'_outstage9numsDBonly.mp4',which = haveSubs )
#putTubesOnVid(DBtubes, vidbase+'_outstage9numsDBsubtubes.mp4', True)


#Now we want to get a motion score for each subtube connection WITHIN TUBE

subtubes = []

for h in haveSubs:
    T = deepcopy(DBtubes[h])
    labels = labelList[h]
    splittube=[]
    A = set(labels)
    if -1 in A:
        A.remove(-1)
    for l in A:
        st=[T[ind] for ind in range(0,len(labels)) if labels[ind]==l]
        splittube.append(st)
    
   
    splittube.sort(key = lambda x: x[0][0])
    subtubes.append(splittube)
    #SUBTUBES NEED TO BE IN FRAME ORDER 
    
#Populated subtubes [[tube in sections], [another one], [[like],[this]]   ]
#Subtubes var goes with haveSubs... same length and haveSubs is the index for the matching tube
#SubTubes (capital T) just means i removed the "outlier" ones from the var

#subTubes = [[st for st in splitT if len(st)>=10] for splitT in subtubes]
subTubes = subtubes
stsizes = [[len(st) for st in splitT] for splitT in subTubes]
stscores= [[0 for st in splitT ] for splitT in subTubes]
print(stsizes)
print(stscores)

for st in stsizes:
    mainSTindex = np.argmax(st)
    thisindex = mainSTindex
    stscores[stsizes.index(st)][mainSTindex] = -1
    tubenum = haveSubs[stsizes.index(st)]
    #IF TUBENUM IS IN HAVEGLIDES... DON'T DO ANYTHING FOR MOTION. DO SOMETHING WITH STSCORES TO FORCE SPLIT
    if tubenum in haveGlides:
        #stscores[stsizes.index(st)] = [-1, -0.5] #force a split with negative num and skip motion stuff
        #subTubes[stsizes.index(st)] = [[],[]]
        #subTubes[stsizes.index(st)][0] = [DBtubes[tubenum][ind] for ind in range(0,len(DBtubes[tubenum])) if labelList[tubenum][ind]==0 ]
        #subTubes[stsizes.index(st)][1] = [DBtubes[tubenum][ind] for ind in range(0,len(DBtubes[tubenum])) if labelList[tubenum][ind]==1 ]
        continue
        
    print()
    while len(st)-1 > thisindex:  #if there are more subtubes after
        #Forward motion score
        print("Going forward on tube ",tubenum,"...")
        thisST = subTubes[stsizes.index(st)][thisindex]
        nextST = subTubes[stsizes.index(st)][thisindex+1]
        R = (thisST[-1][1][1][0]+thisST[-1][1][1][1])/2
        predf = nextST[0][0]
        predX = thisST[-1][2][0]*(thisST[-1][0]-predf) + thisST[-1][1][0][0]
        predY = thisST[-1][2][1]*(thisST[-1][0]-predf) + thisST[-1][1][0][1]
        diffX = abs(nextST[0][1][0][0] - predX)
        diffY = abs(nextST[0][1][0][1] - predY)
        Xscore = math.exp(-(diffX/R))
        Yscore = math.exp(-(diffY/R))
        print("X score is:", Xscore)
        print("Y score is:", Yscore)
        
        #FIND A WAY TO GET THE COSINE OF THE ANGLE BETWEEN THE SUBTUBE VECTORS
        thisT = np.array([e[0] for e in thisST])
        thisX = np.array([e[1][0][0] for e in thisST])
        thisY = np.array([e[1][0][1] for e in thisST])
        nextT = np.array([e[0] for e in nextST])
        nextX = np.array([e[1][0][0] for e in nextST])
        nextY = np.array([e[1][0][1] for e in nextST])

        thisXofT = np.polyfit(thisT,thisX,1)
        thisYofT = np.polyfit(thisT,thisY,1)
        nextXofT = np.polyfit(nextT,nextX,1)
        nextYofT = np.polyfit(nextT,nextY,1)
        
        #print(thisXofT,nextXofT)
        #print(thisYofT,nextYofT)
        Xangle = np.arctan(abs((nextXofT[0]-thisXofT[0]) / (1 + nextXofT[0]*thisXofT[0]))) 
        Xcos = np.cos(Xangle)
        Yangle = np.arctan(abs((nextYofT[0]-thisYofT[0]) / (1 + nextYofT[0]*thisYofT[0]))) 
        Ycos = np.cos(Yangle)
        print("X cosine score", Xcos)
        print("Y cosine score", Ycos)
        
        finalscore = (max(Xscore,Yscore) + (Xcos+Ycos)/2)/2
        if len(nextST)<=20:
            finalscore = 1
        stscores[stsizes.index(st)][thisindex+1] = finalscore
        print("FINAL:", finalscore)
        
        
        
        thisindex=thisindex+1
        
    thisindex = mainSTindex    
    while thisindex>0:  #if there are more subtubes before
        #Backwards motion score
        print("Going backward on tube", tubenum ,"...")
        thisST = subTubes[stsizes.index(st)][thisindex]
        lastST = subTubes[stsizes.index(st)][thisindex-1]
        R = (thisST[0][1][1][0]+thisST[0][1][1][1])/2
        predf = lastST[-1][0]
        predX = thisST[0][1][0][0] - thisST[0][2][0]*abs(thisST[0][0]-predf) 
        predY = thisST[0][1][0][1] - thisST[0][2][1]*abs(thisST[0][0]-predf) 
        diffX = abs(lastST[-1][1][0][0] - predX)
        diffY = abs(lastST[-1][1][0][1] - predY)
        Xscore = math.exp(-(diffX/R))
        Yscore = math.exp(-(diffY/R))
        print("X score is:", Xscore)
        print("Y score is:", Yscore)
        
        
        #FIND A WAY TO GET THE COSINE OF THE ANGLE BETWEEN THE SUBTUBE VECTORS
        thisT = np.array([e[0] for e in thisST])
        thisX = np.array([e[1][0][0] for e in thisST])
        thisY = np.array([e[1][0][1] for e in thisST])
        lastT = np.array([e[0] for e in lastST])
        lastX = np.array([e[1][0][0] for e in lastST])
        lastY = np.array([e[1][0][1] for e in lastST])

        thisXofT = np.polyfit(thisT,thisX,1)
        thisYofT = np.polyfit(thisT,thisY,1)
        lastXofT = np.polyfit(lastT,lastX,1)
        lastYofT = np.polyfit(lastT,lastY,1)
        
        #print(lastXofT,thisXofT)
        #print(lastYofT,thisYofT)
        Xangle = np.arctan(abs((lastXofT[0]-thisXofT[0]) / (1 + lastXofT[0]*thisXofT[0]))) 
        Xcos = np.cos(Xangle)
        Yangle = np.arctan(abs((lastYofT[0]-thisYofT[0]) / (1 + lastYofT[0]*thisYofT[0]))) 
        Ycos = np.cos(Yangle)
        print("X cosine score", Xcos)
        print("Y cosine score", Ycos)
        
        finalscore = (max(Xscore,Yscore) + (Xcos+Ycos)/2)/2
        if len(lastST)<=20:
            finalscore=1
        stscores[stsizes.index(st)][thisindex-1] = finalscore
        print("FINAL:", finalscore)
        
        thisindex = thisindex-1

        #FIND A WAY TO GET THE COSINE OF THE ANGLE BETWEEN THE SUBTUBE VECTORS
        #THEN COMBINE THAT WITH THE BEST X/YSCORE FROM ABOVE TO GET FINAL MOTION SCORE
print(haveSubs)       
print(stscores)


#Putting subtubes back so they can be viewed (DBtubes2)

subInds = []
alpha = 0.8

for i in range(0,len(subTubes)): #for each split tube
    scores = stscores[i]
    startind = scores.index(-1)
    indLists = [[startind]]
    thisind = startind
    while len(scores)-1 > thisind:  #Forward Loop
        if scores[thisind+1] > alpha:
            A = [l for l in indLists if thisind in l][0] #the one with thisind
            A.append(thisind+1)
        else:
            indLists.append([thisind+1])
        thisind=thisind+1
        
    thisind = startind    
    while thisind>0: #Backward Loop
        if scores[thisind-1] > alpha:
            A = [l for l in indLists if thisind in l][0] #the one with thisind
            A.append(thisind-1)
        else:
            indLists.append([thisind-1])
        thisind = thisind-1

    subInds.append(indLists)

print(subInds)
newsubTubes = []
for st in range(0,len(subInds)):
    splitT = []
    for group in subInds[st]:
        #thisST = subTubes[st][thisindex]
        A = []
        for g in group:
            thisST = subTubes[st][g]
            A.extend(thisST)
        splitT.append(A)

    newsubTubes.append(splitT)
    
    
#Change color based on subtube    
DBtubes2 = []
for i in range(0,len(DBtubes)):
    if i not in haveSubs:
        DBtubes2.append(DBtubes[i])
    else:
        T = []
        RGB0 = [random.randint(0,255), random.randint(0,255), random.randint(0,255)]
        RGB1 = [random.randint(0,255), random.randint(0,255), random.randint(0,255)]
        RGB2 = [random.randint(0,255), random.randint(0,255), random.randint(0,255)]
        RGB3 = [random.randint(0,255), random.randint(0,255), random.randint(0,255)]
        stindex = haveSubs.index(i)
        for coll in newsubTubes[stindex]: #for each collection of ellipses
            groupnum = newsubTubes[stindex].index(coll)
            for ell in coll:
                if groupnum==0:
                    ell[3] = RGB0
                elif groupnum==1:
                    ell[3] = RGB1
                elif groupnum==2:
                    ell[3] = RGB2
                else:
                    ell[3] = RGB3
            T.extend(coll)
        DBtubes2.append(T)
            
for h in haveSubs:
    grouping = subInds[haveSubs.index(h)]
    print(grouping)
    for g in grouping:
        labelList[h] = [x if (x==min(g) or x not in g) else min(g) for x in labelList[h] ]
                
 
 # Just finished doing the motion consideration for subtubes
    #Spatial consideration to come....

    

[2, 7, 8, 10, 15, 18, 25, 27, 30, 32]

Has subtubes: [2, 7, 8, 10, 15, 18, 25, 27, 30, 32]
Has glides: [4, 7, 10, 18, 25, 27]
Has steadys: [0, 11, 24]
0.01737619461337967 on tube 0  subtube 0
0.3055555555555556 on tube 0  subtube 1
0.12571428571428572 on tube 0  subtube 2
0.03125 on tube 32  subtube 0
0.0 on tube 32  subtube 1
0.09411764705882353 on tube 32  subtube 2
0.024271844660194174 on tube 2  subtube 0
0.0 on tube 2  subtube 1
0.0 on tube 2  subtube 2
0.0 on tube 2  subtube -1
0.0 on tube 7  subtube 0
0.0 on tube 7  subtube 1
0.0 on tube 7  subtube 2
0.0 on tube 7  subtube -1
0.0 on tube 8  subtube 0
0.09090909090909091 on tube 8  subtube 1
0.0 on tube 10  subtube 0
0.01932367149758454 on tube 10  subtube 1
0.30042918454935624 on tube 10  subtube 2
0.9365079365079365 on tube 10  subtube 3
Percent steady= 0.9365079365079365 on tube 10  subtube 3 starting at 440
0.0 on tube 11  subtube 0
0.3235294117647059 on tube 11  subtube 1
1.0 on tube 11  subtube 2
Percent steady= 1.0 on tube

In [ ]:
#print(labelList[0])
#print(save1)
#print()
#print(labelList[1])
#print(save2)
#print()
#print(labelList[2])
#print(save3)

plotSubTubes(tubes,labelList,which=[2])

In [32]:

#putSomeTubesOnVid(DBtubes2, vidbase+'_outstage9.5motionA08NS.mp4',which = haveSubs )
#putTubesOnVid(DBtubes2, vidbase+'_outstage9.5motionA08ALL.mp4', True)
V = readvid(vidbase+'.mp4')


# Playing with SURF features to see if they can be compared...
# Want to check features within the ellipse at the end of subtube and start of the next... for now

#For now it'll be two frames from the same tube

for l in range(0,len(labelList)):
    if not (len(DBtubes2[l]) == len(labelList[l])):
        print("weird",l)
        A = [DBtubes[l][ind] for ind in range(0,len(labelList[l])) if labelList[l][ind]==-1 ]
        DBtubes2[l].extend(A)
        
    

for t in DBtubes2:
    t.sort(key = lambda x: x[0])
for s in subInds:
    s.sort(key = lambda x: min(x))

subInds2=deepcopy(subInds)
surf = cv2.xfeatures2d.SURF_create()
orb = cv2.ORB_create()

feats = []

for s in range(0,len(subInds)):
    if len(subInds[s])>1:
        tubenum = haveSubs[s]
        splitT = []
        print("Tube:", tubenum)
        for st in subInds[s]: #for each subtube
            ST = [DBtubes2[tubenum][i] for i in range(0,len(labelList[tubenum])) if labelList[tubenum][i] in st]
            splitT.append(ST)
            DaMatches = []

        
        #for each thing in splitT if i+1 < len
        for i in range(0,len(splitT)-1):
            laste = splitT[i][-1]
            nexte = splitT[i+1][0]
            bbox = getBoxFromEllipse(laste)
            bbox2 = getBoxFromEllipse(nexte)
            img = V[int(laste[0])][bbox[1]:bbox[1]+bbox[3], bbox[0]:bbox[0]+bbox[2]]
            img2 = V[int(nexte[0])][bbox2[1]:bbox2[1]+bbox2[3] , bbox2[0]:bbox2[0]+bbox2[2]]
            KP1,desc1 = orb.detectAndCompute(img,None)
            KP2,desc2 = orb.detectAndCompute(img2,None)
            print(len(KP1), len(KP2))
           
            
            if min(len(KP1),len(KP2)) > 5:
                # Brute Force Matching
                bf = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=True)
                #bf = cv2.BFMatcher(cv2.NORM_HAMMING)
                matches = bf.match(desc1, desc2)

                print("Matches",len(matches))
                spatialscore = len(matches)/ min(len(KP1),len(KP2))
                print(spatialscore)
                if spatialscore> 0.5:
                    print("MATCH.")
                    print()
                    A = deepcopy(subInds[s])
                    A[i].extend(A[i+1])
                    del A[i+1]
                    subInds2[s]=A
                    
                    
                matches = sorted(matches, key = lambda x:x.distance)
                DaMatches.append(matches)
                # Draw first 20 matches.
                img3 = cv2.drawMatches(img,KP1,img2,KP2,matches[:5],None, flags=2)

                #cv2.imshow("img3",img3)
                #cv2.waitKey(0)
                #cv2.destroyAllWindows()
                
            else:
                #you can't pull together 5 features
                print("No match.")
                print()
            
# subInds2 is the var that tells us the splits when there's a subtube.

tubesToAdd = []
for i in range(0,len(DBtubes2)):
    if i in haveSubs:
        splits = subInds2[haveSubs.index(i)]
        if len(splits)<=1:
            #Ya Good.
            print("no subtubes here..")
        else:
            toDel=[]
            for s in range(1,len(splits)):
                ST = [DBtubes2[i][ind] for ind in range(0,len(labelList[i])) if labelList[i][ind] in splits[s] ]
                tubesToAdd.append(ST)
                toDel.append([e for e in DBtubes2[i] if e not in ST])
            DBtubes2[i]=[e for e in DBtubes[i] if e not in toDel]
        
        
DBtubes2.extend(tubesToAdd)
#Fix color
for t in DBtubes2:
    RGB = t[0][3]
    for e in t:
        e[3] = RGB

# No ellipse can be in multiple tubes... fix.
L = deepcopy(DBtubes2)
for i in range(0, len(L)):
    for e in L[i]:
        A = [ell[1] for j in range(i,len(L)) for ell in L[j] if (ell[1]==e[1]) and(j!=i)]
        if A:
            #print("Tube",i,"and",j,"share this ellipse.")
            #print(A)
            L[i] = [e for e in L[i] if e[1] not in A]
DBtubes2=L
    

        
putTubesOnVid(DBtubes2, vidbase+'_outstage9.5reattachedA.mp4', True)

cv2.destroyAllWindows()


<class 'list'>
weird 2
weird 7
weird 10
weird 18
weird 25
weird 27
weird 30
Tube: 7
205 137
Matches 89
0.6496350364963503
MATCH.

0 0
No match.

Tube: 8
0 0
No match.

Tube: 10
62 39
Matches 26
0.6666666666666666
MATCH.

0 0
No match.

Tube: 15
0 0
No match.

Tube: 18
68 97
Matches 45
0.6617647058823529
MATCH.

Tube: 25
213 111
Matches 76
0.6846846846846847
MATCH.

Tube: 27
213 127
Matches 83
0.6535433070866141
MATCH.

Tube: 32
0 0
No match.

no subtubes here..
no subtubes here..
no subtubes here..
no subtubes here..
no subtubes here..
FRAME 100 / 1375
FRAME 200 / 1375
FRAME 300 / 1375
FRAME 400 / 1375
FRAME 500 / 1375
FRAME 600 / 1375
FRAME 700 / 1375
FRAME 800 / 1375
FRAME 900 / 1375
FRAME 1000 / 1375
FRAME 1100 / 1375
FRAME 1200 / 1375
FRAME 1300 / 1375


In [ ]:

#--------------------------IDENTIFY---------------------------------
A = 10
tubes = deepcopy(DBtubes2)
hasv0 = []

for i in range(len(tubes)):
    print("Tube", i)
    #get avg location of 1st and last 10 frames. (maybe also middle)
    firstXs = [e[1][0][0] for e in tubes[i] if tubes[i].index(e)<A]
    firstYs = [e[1][0][1]-e[2][1] for e in tubes[i] if tubes[i].index(e)<A]
    X1 = sum(firstXs)/A
    Y1 = sum(firstYs)/A
    X2 = tubes[i][round(len(tubes[i])/2)][1][0][0]
    Y2 = tubes[i][round(len(tubes[i])/2)][1][0][1]
    lastXs = [e[1][0][0] for e in tubes[i] if tubes[i].index(e)>=len(tubes[i])-A]
    lastYs = [e[1][0][1]-e[2][1] for e in tubes[i] if tubes[i].index(e)>=len(tubes[i])-A]
    X3 = sum(lastXs)/A
    Y3 = sum(lastYs)/A
    jump1 = math.sqrt((X2-X1)**2 + (Y2-Y1)**2)
    jump2 = math.sqrt((X3-X2)**2 + (Y3-Y2)**2)
    deltaX= (sum(firstXs)/A - sum(lastXs)/A) 
    deltaY= (sum(firstYs)/A - sum(lastYs)/A) 
    v0score = (7*min(abs(deltaX),abs(deltaY)) + max(abs(deltaX),abs(deltaY)))/len(tubes[i]) 
    #v0score = (jump1+jump2)/len(tubes[i])
    #print("jumps", jump1, jump2)
    print("Score",v0score)
    print(deltaX)
    print(deltaY)
    if (v0score < 1) and ( max(abs(deltaX),abs(deltaY))<50):
        hasv0.append(i)
        #Change the tube to white for now...
        #for e in tubes[i]:
        #    e[3] = [0,0,0]
            
#putTubesOnVid(tubes, vidbase+'_outstage10v0.mp4', True)

##--------------------------GROUP---------------------------------
if hasv0:
    avginfo = [] #matches up with hasv0
    v0groups = [[hasv0[0]]]

    #for each in hasv0... gimme the avg position and max maj axis
    for h in hasv0:
        T = tubes[h]
        Txs = [e[1][0][0] for e in T]
        Tys = [e[1][0][1] for e in T]
        Tmajs = [e[1][1][1] for e in T]
        avginfo.append([sum(Txs)/len(Txs), sum(Tys)/len(Tys), sum(Tmajs)/len(Tmajs) ])
    print(hasv0)

    for h in hasv0[1:]:
        i = hasv0.index(h)

        belongs = False
        for g in v0groups:
            dist = math.sqrt((avginfo[i][0]- avginfo[hasv0.index(g[0])][0])**2 + (avginfo[i][1]-avginfo[hasv0.index(g[0])][1])**2)
            if (dist < avginfo[hasv0.index(g[0])][2]):
                g.append(h)
                belongs = True
                break
        if not belongs:
            v0groups.append([h])

    print(v0groups)

##----------------------------COMBINE---------------------------------


    for g in v0groups:

        supertube = []
        x0 = avginfo[hasv0.index(g[0])][0]
        y0 = avginfo[hasv0.index(g[0])][1]
        phi = tubes[g[0]][0][1][2] 
        mins = [e[1][1][0] for k in g for e in tubes[k] ]
        maxs = [e[1][1][1] for k in g for e in tubes[k] ]
        A = max(mins)
        B = max(maxs)
        fs = [e[0] for k in g for e in tubes[k] ]
        F0 = int(min(fs))
        F1 = int(max(fs))

        for f in range(F0,F1+1):
            ell = [f, ((x0,y0), (A,B), phi), [0,0], [255,255,255] ]
            supertube.append(ell)

        tubes.append(supertube)
    

    
putTubesOnVid(tubes, vidbase+'_outstage10v0grouped.mp4', True)    

In [ ]:
#Now we want to identify tubes that have a long-term velocity of 0

#Maybe we can trigger it by seeing how much the top of the ellipse moved in comparison to how long it is

for i in range(len(tubes)):
    print("Tube", i)
    movedsum = 0
    for j in range(1,len(tubes[i])):
        if j%20==0:
            e = tubes[i][j]
            moved = ((tubes[i][j][1][0][1]-tubes[i][j][2][1]) - (tubes[i][j-10][1][0][1]-tubes[i][j-10][2][1]))**2 + (tubes[i][j][1][0][0]-tubes[i][j-10][1][0][0])**2
            movedsum = movedsum+moved
    
    print("It moved", movedsum/len(tubes[i]), "pixels per frame")
    print()
    
    




In [ ]:
print(haveSteadys)

In [ ]:
'''#Want to remove any subtube (and anything after it) that is steady
#Only considering things with subtubes
#THIS BLOCK IS LOOKING REEEAL "DELETE IF THE ONE ABOVE WORKS BETTER"-ISH


for h in haveSubs:
    STlabels = labelList[h]
    T = DBtubes[h]
    for l in set(STlabels): #for each subtube
        ST = [T[ind] for ind in range(0,len(STlabels)) if STlabels[ind]==l] #actual subtube
        #Check for a 2 frame gap without 5 pix of movement in any dim..
        #Get score for how often that happens...More than 20% is bad
        count=0
        steadycount=0
        for i in range(0,len(ST)):
            for j in range(i,len(ST)):
                count=count+1
                if (abs(ST[j][0]-ST[i][0])>=2) and (abs(ST[j][1][0][0] - ST[i][1][0][0])<3) and (abs(ST[j][1][0][1] - ST[i][1][0][1])<3):
                    steadycount=steadycount+1
                    
                    #steadyFlag = True
                    #print("On tube",h,"delete subtube", l)            
        
        if steadycount/count > 0.2:
            firstind = list(STlabels).index(l)
            print(steadycount/count, "on tube",h," subtube", l, "starting at", firstind)
            #labelList[h] = [STlabels[ind] for ind in range(0,len(STlabels)) if STlabels[ind]!=l]
            #DBtubes[h] = [T[ind] for ind in range(0,len(T)) if STlabels[ind]!=l]
            labelList[h] = STlabels[0:firstind]
            DBtubes[h] = T[0:firstind]
            break

print(len(labelList))
print(len(DBtubes))
L = deepcopy(DBtubes)
            
#Remove empty tubes... clear them from haveSubs and labelList also
toDel = [i for i in range(0,len(DBtubes)) if len(DBtubes[i])==0]
toDel.sort(reverse=True)
print("To delete:", toDel)
for d in toDel:
    haveSubs = [h-1  if h>d else h for h in haveSubs]
    haveSubs.remove(d)
    haveGlides = [h-1 if h>d else h for h in haveGlides]
    if d in haveGlides:
        haveGlides.remove(d)
    
DBtubes = [t for t in DBtubes if DBtubes.index(t) not in toDel]
labelList = [labelList[i] for i in range(0,len(labelList)) if i not in toDel]

print(haveSubs)
putSomeTubesOnVid(DBtubes, vidbase+'_outstage9numsNoSteadys.mp4',which = haveSubs )
    '''

In [ ]:
#Readjusting the velocities based on a sliding window
L = deepcopy(DBtubes2)
Vgap = 3  #calculating velocities across +/- Vgap 
for t in L:
    for i in range(0,len(t)): #for each ellipse
        close = [e for e in t if abs(e[0] - t[i][0])<= Vgap]
        starter = close[0]
        ender = close[-1]
        fgap = ender[0]-starter[0]
        Vx = (ender[1][0][0] - starter[1][0][0]) / (abs(fgap)+1)
        Vy = (ender[1][0][1] - starter[1][0][1]) / (abs(fgap)+1)
        t[i][2] = [Vx, Vy]


# Recognize glides and steadys within a tube (and maybe split)
tubestoAdd=[]
for i in range(0,len(L)):
    scount=0
    gcount=0
    j=0
    slist=[]
    glist=[]
    while j+1 < len(L[i]):
        e= L[i][j]
        startf=e[0]
        if not( abs(e[2][0])>0.1 and abs(e[2][1])>0.1 ):
            scount=scount+1
        else:
            scount=0
        
        if abs(e[2][0]-L[i][j-1][2][0])<0.1 and abs(e[2][1]-L[i][j-1][2][1])<0.1 :
            gcount=gcount+1
        else:
            gcount=0
            
        if scount>30:
            slist.append(startf)
        if gcount>15:
            #print("Gliding at tube", i)
            glist.append(startf)
            
        j=j+1
        
    print("Tube", i)
    print(slist)
    print(glist)
    if len(slist)>=10:
        L[i] = [e for e in L[i] if (e[0]<min(slist)-30)]
        A = [e for e in L[i] if e[0]>max(slist) ]
        tubesToAdd.append(A)
    if len(glist)>=10:
        L[i] = [e for e in L[i] if (e[0]<min(glist)-15) ]
        A = [e for e in L[i] if e[0]>max(glist)]
        tubesToAdd.append(A)
        
L.extend(tubesToAdd)
temporalcutoff = 20
L = [p for p in L if len(p)> temporalcutoff]        
#newpaths = [p for p in L if (p[1][0]-p[0][0] < temporalcutoff)]

        
    
    
putTubesOnVid(L, vidbase+'_outstage9.5reattached2.mp4', True)

In [ ]:
A = [DBtubes2[1][i] for i in range(0,len(labelList[1])) if labelList[1][i] in st]
print(A)



In [ ]:
# USING THE DETECTOR TO FIND 'BREAKS' IN CONTINUITY
#Sorting again... 
for t in tubes:
    t.sort(key = lambda x: x[0])
    
#Readjusting the velocities based on a sliding window
L = deepcopy(tubes)
Vgap = 5  #calculating velocities across +/- Vgap 
for t in L:
    for i in range(0,len(t)): #for each ellipse
        close = [e for e in t if abs(e[0] - t[i][0])<= Vgap]
        starter = close[0]
        ender = close[-1]
        fgap = ender[0]-starter[0]
        Vx = (ender[1][0][0] - starter[1][0][0]) / (abs(fgap)+1)
        Vy = (ender[1][0][1] - starter[1][0][1]) / (abs(fgap)+1)
        t[i][2] = [Vx, Vy]

        
switchtrack = []
checkgap = 30
for t in L:
    for f in range(t[0][0], t[-1][0]):
        Vbefore = [e[2] for e in t if (f-e[0]<checkgap) and (f-e[0]>0)]
        Vafter = [e[2] for e in t if (e[0]-f<checkgap) and (e[0]-f>0)]
        if not(Vbefore) or not(Vafter):
            continue
        Vxbef = np.mean(np.array(Vbefore),axis=0)[0]
        Vxaft = np.mean(np.array(Vafter),axis=0)[0]
        Vybef = np.mean(np.array(Vbefore),axis=0)[1]
        Vyaft = np.mean(np.array(Vafter),axis=0)[1]
        score = 0
        if (Vxaft/Vxbef < 0) and abs(Vxaft-Vxbef)>1 and (f>=t[0][0]+checkgap):  #dir switch in x
            score=score+1
        if (Vyaft/Vybef < 0) and abs(Vyaft-Vybef)>1 and (f>=t[0][0]+checkgap):  #dir switch in y
            score=score+1
        if score>0:
            loc = [e[1][0] for e in t if e[0]==f][0]
            strack = (f,(loc[0],loc[1],50,50),score,L.index(t))
            switchtrack.append(strack)
            #print("Break in tube ", L.index(t))
            
            
        

putMarksOnVid([switchtrack],vidbase+'_outstage8numsNEW2.mp4',vidbase+'_outstage9ShiftMarks.mp4')

In [ ]:
# Stage 9 Vibes (Tracking on both sides of the tube)-------------------------------------

#Sorting again... 
for t in tubes:
    t.sort(key = lambda x: x[0])

L = deepcopy(tubes)
tracks = []

for t in L:
    startf = t[0][0] + 3
    endf = t[-1][0] - 3
    starttrack = []
    endtrack = []
    
    if startf>10:
        #Do KCF tracking in front of this tube (till frame 10)
        startE = t[0]
        tracker = cv2.TrackerMedianFlow_create()
        bbox = getBoxFromEllipse(startE)
        ok = tracker.init(invid[int(startf)], bbox)
        starttrack.append((startf, bbox))
        k=1
        while ok and (startf-k > 10):
        
        ok, bbox = tracker.update(invid[int(startf-k)])
            if ok:
                starttrack.append((startf-k, bbox))
            k=k+1
                   
        
    if endf<(F-10):
        #Do KCF tracking at end of this tube (till frame F-10)
        startE = t[-1]
        tracker = cv2.TrackerMedianFlow_create()
        bbox = getBoxFromEllipse(startE)
        ok = tracker.init(invid[int(endf)], bbox)
        endtrack.append((endf, bbox))
        k=1
        while ok and (endf + k < F-10):
            ok, bbox = tracker.update(invid[int(endf+k)])
            if ok:
                endtrack.append((endf+k, bbox))
            k=k+1
            
    tracks.append(starttrack)
    tracks.append(endtrack)
    
    


In [33]:
writeTubelist(vidbase+'_s9Tubelist.csv', DBtubes2)
#DBtubes2 = readTubelist(vidbase+'_s9Tubelist.csv')
#tubes = readTubelist('int6_s8Tubelist.csv')


In [34]:
#SURF/ORB play
# Big merge vibes
#img1 = cv2.imread("wheelpic1.jpg")
#img2 = cv2.imread("wheelpic2.jpg")

useorb = True #detector choice. SURF or ORB
useflann = 1 #matcher choice. 0=brute force matcher, 1=FLANNforSURF, other=FLANNforORB

def compareImg(img1,img2,useorb=True,useflann=0):
    
    if useorb:
        #-- Step 1A: Detect the keypoints using ORB Detector, compute the descriptors
        detector = cv2.ORB_create()
        keypoints1, descriptors1 = detector.detectAndCompute(img1, None)
        keypoints2, descriptors2 = detector.detectAndCompute(img2, None)
    else:
        #-- Step 1B: Detect the keypoints using SURF Detector, compute the descriptors
        minHessian = 500 #big number = more 'choosy'
        detector = cv2.xfeatures2d_SURF.create(hessianThreshold=minHessian)
        keypoints1, descriptors1 = detector.detectAndCompute(img1, None)
        keypoints2, descriptors2 = detector.detectAndCompute(img2, None)

        
    #Check if there are keypoints
    #print("Num KPS:", len(keypoints1), len(keypoints2))
    if len(keypoints1)==0 or len(keypoints2)==0:
        #If not, do a direct image comparison
        #print("Don't see any KPs here buddy.")
        newH = round((img1.shape[0]+img2.shape[0])/2)
        newW = round((img1.shape[1]+img2.shape[1])/2)
        newimg1 = cv2.resize(img1, dsize=(newW, newH), interpolation=cv2.INTER_CUBIC)
        newimg2 = cv2.resize(img2, dsize=(newW, newH), interpolation=cv2.INTER_CUBIC)
        #print("NORM:", np.linalg.norm( np.subtract(newimg1,newimg2)) )
        res = np.linalg.norm( np.subtract(newimg1,newimg2))
        
    else:
        if useflann==0:
            #-- Step 2A: Matching descriptor vectors with a Brute Forcer
            if useorb:
                bf = cv2.BFMatcher(cv2.NORM_HAMMING,crossCheck=True)
            else:
                bf = cv2.BFMatcher(cv2.NORM_L1,crossCheck=True)
            bfmatches = bf.match(descriptors1,descriptors2)
            bfmatches = sorted(bfmatches, key = lambda x:x.distance)
            result = cv2.drawMatches(img1,keypoints1,img2,keypoints2,bfmatches,None)  
            #print("Num matches:", len(bfmatches))
            #print("Percent = ", len(bfmatches)/len(keypoints1)*100)
            res = len(bfmatches)/len(keypoints1)
            
        elif useflann==1:    
            #-- Step 2B: Matching descriptor vectors with a FLANN based matcher
            # Since SURF is a floating-point descriptor NORM_L2 is used
            matcher = cv2.DescriptorMatcher_create(cv2.DescriptorMatcher_FLANNBASED)
            knn_matches = matcher.knnMatch(descriptors1, descriptors2, 2)

            #-- Filter matches using the Lowe's ratio test
            good_matches = []
            for m,n in knn_matches:
                if m.distance < 0.7 * n.distance:
                    good_matches.append(m)

            img_matches = np.empty((max(img1.shape[0], img2.shape[0]), img1.shape[1]+img2.shape[1], 3), dtype=np.uint8)
            result = cv2.drawMatches(img1, keypoints1, img2, keypoints2, good_matches, img_matches, flags=cv2.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS)
            #print("Num matches:", len(good_matches))
            #print("Percent = ", len(good_matches)/len(keypoints1)*100)
            res = len(good_matches)/len(keypoints1)
            
        #-- Show detected matches
        #cv2.imshow('Good Matches', result)
        #cv2.waitKey(0)
        
        

    cv2.destroyAllWindows()
    return res
    
def getImgFromEllipse(E):
    B = getBoxFromEllipse(E)
    imgA = V[int(E[0])][B[1]:B[1]+B[3],B[0]:B[0]+B[2]]
    
    #cv2.imshow('Img from ellipse', imgA)
    #cv2.waitKey(0)
    #cv2.destroyAllWindows()
    return imgA

def compareImgFromEllipse(E1,E2=None,useorb=True,useflann=0):
    img1 = getImgFromEllipse(E1)
    if not(E2):
        img2 = V[int(E1[0])]
        res = compareImg(img1,img2,useorb,useflann)
        return res
    else:
        img2 = getImgFromEllipse(E2)
        res = compareImg(img1,img2,useorb,useflann)
        return res
            
def getMotionScores(T1, T2):
    #for now... returns
    # [Xscore,Yscore,Locscore,Xcos,Ycos,Vxfir,Vxsec,Vyfir,Vysec]
    if (T1[round(len(T1)/2)][0] < T2[round(len(T2)/2)][0]):
        isT2After = True
    else:
        isT2After = False
        
    #Spatial prediction (going in A frames deep either side)
    A = 10
    predf = T2[A][0]
    R = (T1[-A][1][1][0]+T1[-A][1][1][1])/2
    if predf in [e[0] for e in T1]:
        #Unlikely but if so, use this as the prediction
        fgap=1
        predX = [e for e in T1 if e[0]==predf][0][1][0][0]
        predY = [e for e in T1 if e[0]==predf][0][1][0][1]
    else:
        fgap = abs(T2[A][0]-T1[-A][0])
        predX = T1[-A][2][0]*fgap + T1[-A][1][0][0]
        predY = T1[-A][2][1]*fgap + T1[-A][1][0][1]
    diffX = abs(T2[A][1][0][0] - predX)
    diffY = abs(T2[A][1][0][1] - predY)
    diff = math.sqrt(diffX**2 +  diffY**2)
    Xtrav = abs(T1[-1][1][0][0] - T1[0][1][0][0])
    Ytrav = abs(T1[-1][1][0][1] - T1[0][1][0][1])
    ExpX = T1[-A][2][0] * fgap
    ExpY = T1[-A][2][1] * fgap
    #Xscore = math.exp(-(diffX/math.sqrt(ExpX**2+ExpY**2)))
    #Yscore = math.exp(-(diffY/math.sqrt(ExpX**2+ExpY**2)))
    #Locscore = math.exp(-(diff/math.sqrt(ExpX**2+ExpY**2)))
    Xscore = math.exp(-(diffX/math.sqrt(Xtrav**2+Ytrav**2)))
    Yscore = math.exp(-(diffY/math.sqrt(Xtrav**2+Ytrav**2)))
    Locscore = math.exp(-(diff/math.sqrt(Xtrav**2+Ytrav**2)))
    #print("X score is:", diffX, Xscore)
    #print("Y score is:", diffY, Yscore)
    #print("Loc score is:",diff,Locscore)
    
    
    #Cosine directional alignment
    thisT = np.array([e[0] for e in T1])
    thisX = np.array([e[1][0][0] for e in T1])
    thisY = np.array([e[1][0][1] for e in T1])
    nextT = np.array([e[0] for e in T2])
    nextX = np.array([e[1][0][0] for e in T2])
    nextY = np.array([e[1][0][1] for e in T2])
    thisXofT = np.polyfit(thisT,thisX,1)
    thisYofT = np.polyfit(thisT,thisY,1)
    nextXofT = np.polyfit(nextT,nextX,1)
    nextYofT = np.polyfit(nextT,nextY,1)
    Xangle = np.arctan(abs((nextXofT[0]-thisXofT[0]) / (1 + nextXofT[0]*thisXofT[0]))) 
    Xcos = np.cos(Xangle)
    Yangle = np.arctan(abs((nextYofT[0]-thisYofT[0]) / (1 + nextYofT[0]*thisYofT[0]))) 
    Ycos = np.cos(Yangle)
    #print("X cosine score", Xcos)
    #print("Y cosine score", Ycos)
    
    #Velocity comparison
    Vx1 = [e[2][0] for e in T1]
    Vy1 = [e[2][1] for e in T1]
    Vx2 = [e[2][0] for e in T2]
    Vy2 = [e[2][1] for e in T2]
    Vxfir = ((sum(Vx1)/len(Vx1)) + (sum(Vx1[-A:])/A))/2
    Vxsec = ((sum(Vx2)/len(Vx2)) + (sum(Vx2[-A:])/A))/2
    Vyfir = ((sum(Vy1)/len(Vy1)) + (sum(Vy1[-A:])/A))/2
    Vysec = ((sum(Vy2)/len(Vy2)) + (sum(Vy2[-A:])/A))/2
    
    #print("Vx matchup", Vxfir, Vxsec)
    #print("Vy matchup", Vyfir, Vysec)
    
    #print("Perc diff Vx:", abs((Vxsec - Vxfir)/Vxfir) )
    #print("Perc diff Vy:", abs((Vysec - Vyfir)/Vyfir) )

    scores = [Xscore, Yscore, Locscore, Xcos, Ycos, Vxfir, Vxsec, Vyfir,Vysec]
    return scores
    

    
    

In [35]:
#Cleaning up



t0 = time.time()

tubes = deepcopy(DBtubes2)
tubes = [tu for tu in tubes if len(tu)!=0]
toDel = []
for t in tubes:
    #if len(t)<30:
    #    print("Tube ",tubes.index(t), "is too short")
    #    toDel.append(t)

    #Check for "inner" tubes and delete
    innercount=0
    for e in t:
        lile = getBoxFromEllipse(e)
        esonframe = [ell for u in tubes for ell in u if ell[0]==e[0]]
        esonframe.remove(e)
        for ell in esonframe:
            bige = getBoxFromEllipse(ell) 
            if (bige[0]<lile[0]) and (lile[0]<bige[0]+bige[2]) and (bige[1]<lile[1]) and (lile[1]<bige[1]+bige[3]) and (lile[2]<bige[2]) and(lile[3]<bige[3]):
                innercount = innercount+1
                break
    if innercount/len(t) > .5:
        print("Tube ",tubes.index(t),"is an inner tube")
        toDel.append(tubes.index(t))
        
    #Check for tube gaps, delete anything after gap    
    if abs(t[-1][0]-t[0][0] -len(t))>1:
        lastf = 0
        print("Tube ", tubes.index(t), "was cleaned up.")
        fs = [e[0] for e in t]
        for i in range(0,len(fs)-1):
            if fs[i+1]-fs[i]==1:
                continue
            else:
                lastf = fs[i]
                break
       # tubes[tubes.index(t)] = [e for e in t if e[0]<=lastf]
        
for t in tubes:
    if len(t)<12:
        print("Tube ",tubes.index(t), "is too short")
        toDel.append(tubes.index(t))

tubes = [tubes[i] for i in range(0,len(tubes)) if i not in toDel]

putTubesOnVid(tubes, vidbase+'_outstage10PREmerge.mp4', True)




#------------------------------Merge suggestions----------------------------

        
#tubes = deepcopy(DBtubes2)
A = 10
nT = len(tubes)
Merges = []


for t1 in range(0,nT):
    
    scorecard = np.empty([nT,9])
    cantMerge = []
    T1 = tubes[t1]
    T1cantmerge = False


    for k in range(0,nT):
        T2 = tubes[k]
        #print("Tube ",k)
        #Check if one is contained by the other, if so can't merge
        if (T2[0][0]>T1[0][0]) and (T2[-1][0]<T1[-1][0]):
            cantMerge.append(k)
            scorecard[k,:] = [0,0,0,0,0,0,0,0,0]
        else:
            scorecard[k,:] = getMotionScores(T1,T2)
    scorecard[t1,:] = [0,0,0,0,0,0,0,0,0]
    
    #See if T1 goes outta bounds (unmergeable)
    #maxR = max([(e[1][1][1]+e[1][1][0])/2 for e in T1])
    maxmaj = max([e[1][1][1] for e in T1])
    avgVx= sum([e[2][0] for e in T1])/len(T1)
    predXs = [e[2][0]*20 + e[1][0][0] + (e[1][1][0]+e[1][1][1])/2  for e in T1]
    predYs = [e[1][0][1] + e[2][1]*5 for e in T1]
    if (max(predXs)>W-40 and avgVx>1) or (min(predXs)<0 and avgVx<-1) or (max(predYs)>H) or (min(predYs)<0):
        print("This tube goes outta bounds, don't merge it.")
        T1cantmerge = True


    #print("Top scorers in Xpred,Ypred,Locscore:")
    k = 3     #top3
    toplists = []
    for s in range(0,3):
        D = scorecard[:,s]
        top = np.argpartition(D,-k)[-k:]
        toplists.append(top)
        #print(top)

    toMerge = []
    for t in range(0,nT):
        if (t in toplists[2]) and (t in toplists[0] or t in toplists[1]):
            toMerge.append(t)

    toMerge = [m for m in toMerge if (m!=t1) and (m not in cantMerge)]
    WC = scorecard[toMerge]
    Scoreboard = np.zeros(len(toMerge))


    # GO THRU THE 5 POINTS! Add to scoreboard and see who wins.
    if not(toMerge):
        T1cantmerge=True
    else:
        # Best Location
        bestloc = np.argmax(WC[:,2])
        Scoreboard[bestloc] = Scoreboard[bestloc] + 1

        # Smallest velocity change
        bestvel = np.argmin((WC[:,6]-WC[:,5])**2 + (WC[:,8]-WC[:,7])**2)
        Scoreboard[bestvel] = Scoreboard[bestvel] + 1

        # Closest lineup
        bestcos = np.argmax((WC[:,3]+WC[:,4])/2 )
        Scoreboard[bestcos] = Scoreboard[bestcos] + 1

        # Best size matchup
        minax = sum([e[1][1][0] for e in T1]) /len(T1)
        majax = sum([e[1][1][1] for e in T1]) /len(T1)
        sizeboard = []
        for m in toMerge:
            thisminax = sum([e[1][1][0] for e in tubes[m]])/len(tubes[m])
            thismajax = sum([e[1][1][1] for e in tubes[m]])/len(tubes[m])
            sizeboard.append((thisminax-minax)**2+(thismajax-majax)**2)
        bestsiz = np.argmin(sizeboard)
        Scoreboard[bestsiz] = Scoreboard[bestsiz] + 1

        # Best spatial matchup 
        pixboard = []
        for m in toMerge:
            E1 = tubes[t1][-A]
            E2 = tubes[m][A]
            res = compareImgFromEllipse(E1,E2,useorb=False,useflann=0)
            pixboard.append(res)
        if pixboard[0] <= 1:
            bestpix = np.argmax(pixboard)
        else:
            bestpix = np.argmin(pixboard)
        Scoreboard[bestpix] = Scoreboard[bestpix] + 1
        print("Spatial = ",bestpix)

    print("SCORE:")
    print("Possible merges", toMerge)
    print("Num of points",Scoreboard)
    print()
    mergeboard = [[toMerge[i],Scoreboard[i]] for i in range(0,len(toMerge)) ]

    # Make sure the remaining tubes are even mergeable
    endT1 = T1[-1][0]
    for m in mergeboard:
        #Gotta start around or after end of T1
        if tubes[m[0]][0][0] < (endT1-A):
            m[1] = -1

        
        
    print(mergeboard)
    mergeboard.sort(key = lambda x: -x[1])
    print(mergeboard)
    print()

    if not(mergeboard) or mergeboard[0][1]<2 or T1cantmerge:
        print("Don't merge tube", t1)
        Merges.append([t1,-1])
    else:
        #Check for min loc score of 0.1
        Lscore = scorecard[mergeboard[0][0],2]
        if Lscore <= 0.1 and mergeboard[0][1]<4: #check for min loc score
            print("The closest thing was kinda far. Don't merge tube", t1)
            Merges.append([t1,-1])
        else:           
            Vs = scorecard[mergeboard[0][0],5:]
            if (Vs[1]/Vs[0]<0 and abs(Vs[1]-Vs[0])>2 and abs(Vs[0])>0.5 and abs(Vs[1])>0.5 )  :
                print("The closest thing was going in the opp direction. Don't merge",t1)
                Merges.append([t1,-1])
            else:
                print("THE WINNER (for tube ",t1,") IS: TUBE", mergeboard[0][0])
                print("with ",mergeboard[0][1],"points in favor.")
                Merges.append([t1,mergeboard[0][0]])
        
        

print(Merges)
sectubes = [m[1] for m in Merges]
two2one = [t for t in range(0,nT) if sectubes.count(t)>1]
toAdd= []
for t in two2one:
    tm = [m for m in Merges if m[1]==t]
    if len(tm)==2:
        A1 = [e[0] for e in tubes[tm[0][0]]]
        A2 = [e[0] for e in tubes[tm[1][0]]]
        overlap = [val for val in A1 if val in A2]
        if len(overlap)>15:
            Merges = [M for M in Merges if M not in tm] 
        else:
            #chain them
            if A1[0]<A2[0]:
                toAdd.append([tm[0][0],tm[1][0],tm[1][1]])
            else:
                toAdd.append([tm[1][0],tm[0][0],tm[1][1]])
        Merges = [M for M in Merges if M not in tm]   
    else:
        Merges = [M for M in Merges if M not in tm] 

if toAdd:
    for ta in toAdd:
        Merges.append(ta)

print(Merges)
mergechains = []
#Turn Merges into chain
for m in Merges:
    if m[1]==-1:
        continue
    else:
        M1s = [mc[-1] for mc in mergechains]
        if m[0] in M1s:
            mergechains[M1s.index(m[0])].append(m[1])
        else:
            mergechains.append(m)
print(mergechains)


toDel = []
for m in mergechains:
    supertube = []
    checkpts = [tubes[t][A][0]  for t in m] + [tubes[t][-A][0]  for t in m]
    checkpts.sort()
    for t in m:
        toDel.append(t)
        supertube.extend(tubes[t])
    supertube.sort(key = lambda x: x[0])
    
    #do supertube colors
    RGB = tubes[m[0]][0][3]
    for e in supertube:
        e[3] = RGB
    
    for thisf in range(int(supertube[0][0]), int(supertube[-1][0])):
        ishere = [e for e in supertube if e[0]==thisf]
        if ishere:
            continue
        else:
            low = thisf
            high = thisf
            while low not in checkpts and low>supertube[0][0]:
                low=low-1
            while high not in checkpts :
                high=high+1
                
            fgap = high-low
            laste = [e for e in supertube if e[0]==low][0]
            nexte = [e for e in supertube if e[0]==high][0]
            
            X = laste[1][0][0] + (nexte[1][0][0]-laste[1][0][0])*((thisf-low)/fgap)
            Y = laste[1][0][1] + (nexte[1][0][1]-laste[1][0][1])*((thisf-low)/fgap)
            smallax = laste[1][1][0] + (nexte[1][1][0]-laste[1][1][0])*((thisf-low)/fgap)
            bigax = laste[1][1][1] + (nexte[1][1][1]-laste[1][1][1])*((thisf-low)/fgap)
            phi = laste[1][2]
            Vx = laste[2][0] + (nexte[2][0]-laste[2][0])*((thisf-low)/fgap)
            Vy = laste[2][1] + (nexte[2][1]-laste[2][1])*((thisf-low)/fgap)
        
            thisslice = [thisf, ((X,Y),(smallax,bigax),phi), [Vx,Vy], RGB]
            supertube.append(thisslice)
            
            
    #Add supertube. subtract old tubes
    tubes.append(supertube)
    
tubes = [T for T in tubes if tubes.index(T) not in toDel]

t1 = time.time()
tfin = t1 - t0
print("Cleaning, merge suggestion, and merge for",vidbase)
print("Total time in sec", (tfin))
print("Time per tube", (tfin/nT))
print("Time per frame", (tfin/F))

putTubesOnVid(tubes, vidbase+'_outstagev10POSTmerge.mp4', True)



    

Tube  35 was cleaned up.
FRAME 100 / 1375
FRAME 200 / 1375
FRAME 300 / 1375
FRAME 400 / 1375
FRAME 500 / 1375
FRAME 600 / 1375
FRAME 700 / 1375
FRAME 800 / 1375
FRAME 900 / 1375
FRAME 1000 / 1375
FRAME 1100 / 1375
FRAME 1200 / 1375
FRAME 1300 / 1375
Spatial =  2
SCORE:
Possible merges [1, 20, 27]
Num of points [1. 1. 3.]

[[1, -1], [20, -1], [27, -1]]
[[1, -1], [20, -1], [27, -1]]

Don't merge tube 0
Spatial =  0
SCORE:
Possible merges [2]
Num of points [5.]

[[2, -1]]
[[2, -1]]

Don't merge tube 1
Spatial =  1
SCORE:
Possible merges [1, 3, 16]
Num of points [2. 1. 2.]

[[1, -1], [3, -1], [16, -1]]
[[1, -1], [3, -1], [16, -1]]

Don't merge tube 2
Spatial =  0
SCORE:
Possible merges [8, 34]
Num of points [3. 2.]

[[8, 3.0], [34, 2.0]]
[[8, 3.0], [34, 2.0]]

THE WINNER (for tube  3 ) IS: TUBE 8
with  3.0 points in favor.
Spatial =  0
SCORE:
Possible merges [1]
Num of points [5.]

[[1, -1]]
[[1, -1]]

Don't merge tube 4
Spatial =  0
SCORE:
Possible merges [11, 14]
Num of points [5. 0.]

[

In [ ]:
print(thisf)
print(checkpts)
print(low)
print(high)
print(supertube[-1][0])
print([e[0] for e in supertube])

In [ ]:
#tubes = deepcopy(DBtubes2)
A = 10
nT = len(tubes)
Merges = []

t0 = time.time()

for t1 in range(0,nT):
    
    scorecard = np.empty([nT,9])
    cantMerge = []
    T1 = tubes[t1]
    T1cantmerge = False


    for k in range(0,nT):
        T2 = tubes[k]
        #print("Tube ",k)
        #Check if one is contained by the other, if so can't merge
        if (T2[0][0]>T1[0][0]) and (T2[-1][0]<T1[-1][0]):
            cantMerge.append(k)
            scorecard[k,:] = [0,0,0,0,0,0,0,0,0]
        else:
            scorecard[k,:] = getMotionScores(T1,T2)
    scorecard[t1,:] = [0,0,0,0,0,0,0,0,0]
    
    #See if T1 goes outta bounds (unmergeable)
    #maxR = max([(e[1][1][1]+e[1][1][0])/2 for e in T1])
    maxmaj = max([e[1][1][1] for e in T1])
    avgVx= sum([e[2][0] for e in T1])/len(T1)
    predXs = [e[2][0]*20 + e[1][0][0] + (e[1][1][0]+e[1][1][1])/2  for e in T1]
    predYs = [e[1][0][1] + e[2][1]*5 for e in T1]
    if (max(predXs)>W-40 and avgVx>1) or (min(predXs)<0 and avgVx<-1) or (max(predYs)>H) or (min(predYs)<0):
        print("This tube goes outta bounds, don't merge it.")
        T1cantmerge = True


    #print("Top scorers in Xpred,Ypred,Locscore:")
    k = 3     #top3
    toplists = []
    for s in range(0,3):
        D = scorecard[:,s]
        top = np.argpartition(D,-k)[-k:]
        toplists.append(top)
        #print(top)

    toMerge = []
    for t in range(0,nT):
        if (t in toplists[2]) and (t in toplists[0] or t in toplists[1]):
            toMerge.append(t)

    toMerge = [m for m in toMerge if (m!=t1) and (m not in cantMerge)]
    WC = scorecard[toMerge]
    Scoreboard = np.zeros(len(toMerge))


    # GO THRU THE 5 POINTS! Add to scoreboard and see who wins.
    # Best Location
    bestloc = np.argmax(WC[:,2])
    Scoreboard[bestloc] = Scoreboard[bestloc] + 1

    # Smallest velocity change
    bestvel = np.argmin((WC[:,6]-WC[:,5])**2 + (WC[:,8]-WC[:,7])**2)
    Scoreboard[bestvel] = Scoreboard[bestvel] + 1

    # Closest lineup
    bestcos = np.argmax((WC[:,3]+WC[:,4])/2 )
    Scoreboard[bestcos] = Scoreboard[bestcos] + 1

    # Best size matchup
    minax = sum([e[1][1][0] for e in T1]) /len(T1)
    majax = sum([e[1][1][1] for e in T1]) /len(T1)
    sizeboard = []
    for m in toMerge:
        thisminax = sum([e[1][1][0] for e in tubes[m]])/len(tubes[m])
        thismajax = sum([e[1][1][1] for e in tubes[m]])/len(tubes[m])
        sizeboard.append((thisminax-minax)**2+(thismajax-majax)**2)
    bestsiz = np.argmin(sizeboard)
    Scoreboard[bestsiz] = Scoreboard[bestsiz] + 1

    # Best spatial matchup 
    pixboard = []
    for m in toMerge:
        E1 = tubes[t1][-A]
        E2 = tubes[m][A]
        res = compareImgFromEllipse(E1,E2,useorb=False,useflann=0)
        pixboard.append(res)
    if pixboard[0] <= 1:
        bestpix = np.argmax(pixboard)
    else:
        bestpix = np.argmin(pixboard)
    Scoreboard[bestpix] = Scoreboard[bestpix] + 1
    print("Spatial = ",bestpix)

    print("SCORE:")
    print("Possible merges", toMerge)
    print("Num of points",Scoreboard)
    print()
    mergeboard = [[toMerge[i],Scoreboard[i]] for i in range(0,len(toMerge)) ]

    # Make sure the remaining tubes are even mergeable
    endT1 = T1[-1][0]
    for m in mergeboard:
        #Gotta start around or after end of T1
        if tubes[m[0]][0][0] < (endT1-A):
            m[1] = -1

        
        
    print(mergeboard)
    mergeboard.sort(key = lambda x: -x[1])
    print(mergeboard)
    print()

    if mergeboard[0][1]<2 or T1cantmerge:
        print("Don't merge tube", t1)
        Merges.append([t1,-1])
    else:
        #Check for min loc score of 0.1
        Lscore = scorecard[mergeboard[0][0],2]
        if Lscore <= 0.1 and mergeboard[0][1]<4: #check for min loc score
            print("The closest thing was kinda far. Don't merge tube", t1)
            Merges.append([t1,-1])
        else:           
            Vs = scorecard[mergeboard[0][0],5:]
            if (Vs[1]/Vs[0]<0 and abs(Vs[1]-Vs[0])>2 and abs(Vs[0])>0.5 and abs(Vs[1])>0.5 )  :
                print("The closest thing was going in the opp direction. Don't merge",t1)
                Merges.append([t1,-1])
            else:
                print("THE WINNER (for tube ",t1,") IS: TUBE", mergeboard[0][0])
                print("with ",mergeboard[0][1],"points in favor.")
                Merges.append([t1,mergeboard[0][0]])
        
        
t1 = time.time()
print("Total time in sec", (t1-t0))


print(Merges)


In [ ]:
print(getMotionScores(tubes[2],tubes[5]))
#print(getMotionScores(tubes[2],tubes[4]))


In [ ]:
print(Merges)
sectubes = [m[1] for m in Merges]
two2one = [t for t in range(0,nT) if sectubes.count(t)>1]
toAdd= []
for t in two2one:
    tm = [m for m in Merges if m[1]==t]
    if len(tm)==2:
        A1 = [e[0] for e in tubes[tm[0][0]]]
        A2 = [e[0] for e in tubes[tm[1][0]]]
        overlap = [val for val in A1 if val in A2]
        if len(overlap)>15:
            Merges = [M for M in Merges if M not in tm] 
        else:
            #chain them
            if A1[0]<A2[0]:
                toAdd.append([tm[0][0],tm[1][0],tm[1][1]])
            else:
                toAdd.append([tm[1][0],tm[0][0],tm[1][1]])
        Merges = [M for M in Merges if M not in tm]   
    else:
        Merges = [M for M in Merges if M not in tm] 

if toAdd:
    for ta in toAdd:
        Merges.append(ta)

print(Merges)
mergechains = []
#Turn Merges into chain
for m in Merges:
    if m[1]==-1:
        continue
    else:
        M1s = [mc[-1] for mc in mergechains]
        if m[0] in M1s:
            mergechains[M1s.index(m[0])].append(m[1])
        else:
            mergechains.append(m)
print(mergechains)


toDel = []
for m in mergechains:
    supertube = []
    checkpts = [tubes[t][A][0]  for t in m] + [tubes[t][-A][0]  for t in m]
    checkpts.sort()
    for t in m:
        toDel.append(t)
        supertube.extend(tubes[t])
    supertube.sort(key = lambda x: x[0])
    
    #do supertube colors
    RGB = tubes[m[0]][0][3]
    for e in supertube:
        e[3] = RGB
    
    for thisf in range(int(supertube[0][0]), int(supertube[-1][0])):
        ishere = [e for e in supertube if e[0]==thisf]
        if ishere:
            continue
        else:
            low = thisf
            high = thisf
            while low not in checkpts:
                low=low-1
            while high not in checkpts:
                high=high+1
                
            fgap = high-low
            laste = [e for e in supertube if e[0]==low][0]
            nexte = [e for e in supertube if e[0]==high][0]
            
            X = laste[1][0][0] + (nexte[1][0][0]-laste[1][0][0])*((thisf-low)/fgap)
            Y = laste[1][0][1] + (nexte[1][0][1]-laste[1][0][1])*((thisf-low)/fgap)
            smallax = laste[1][1][0] + (nexte[1][1][0]-laste[1][1][0])*((thisf-low)/fgap)
            bigax = laste[1][1][1] + (nexte[1][1][1]-laste[1][1][1])*((thisf-low)/fgap)
            phi = laste[1][2]
            Vx = laste[2][0] + (nexte[2][0]-laste[2][0])*((thisf-low)/fgap)
            Vy = laste[2][1] + (nexte[2][1]-laste[2][1])*((thisf-low)/fgap)
        
            thisslice = [thisf, ((X,Y),(smallax,bigax),phi), [Vx,Vy], RGB]
            supertube.append(thisslice)
            
            
    #Add supertube. subtract old tubes
    tubes.append(supertube)
    
tubes = [T for T in tubes if tubes.index(T) not in toDel]

putTubesOnVid(tubes, vidbase+'_outstagev10POSTmerge.mp4', True)

In [ ]:
T1 = tubes[2]
#See if T1 goes outta bounds (unmergeable)
maxR = max([(e[1][1][1]+e[1][1][0])/2 for e in T1])
maxmaj = max([e[1][1][1] for e in T1])
avgVx= sum([e[2][0] for e in T1])/len(T1)
predXs = [e[2][0]*20 + e[1][0][0] + (e[1][1][0]+e[1][1][1])/2  for e in T1]
#predX2s = [e[1][0][0] + maxmaj for e in T1]
predYs = [e[1][0][1] + e[2][1]*5 for e in T1]
print(max(predXs))
if (max(predXs)>W-40 and avgVx>0) or (min(predXs)<50 and avgVx<0) or (max(predYs)>H) or (min(predYs)<0):
    print("This tube goes outta bounds, don't merge it.")
    #print(predX1s)
    print(max(predXs))



In [ ]:
print(getMotionScores(tubes[2],tubes[7]))

In [ ]:
print(scorecard[0,:])

In [ ]:
H

In [ ]:

#FIND A WAY TO GET THE COSINE OF THE ANGLE BETWEEN THE SUBTUBE VECTORS
        thisT = np.array([e[0] for e in thisST])
        thisX = np.array([e[1][0][0] for e in thisST])
        thisY = np.array([e[1][0][1] for e in thisST])
        nextT = np.array([e[0] for e in nextST])
        nextX = np.array([e[1][0][0] for e in nextST])
        nextY = np.array([e[1][0][1] for e in nextST])

        thisXofT = np.polyfit(thisT,thisX,1)
        thisYofT = np.polyfit(thisT,thisY,1)
        nextXofT = np.polyfit(nextT,nextX,1)
        nextYofT = np.polyfit(nextT,nextY,1)
        
        #print(thisXofT,nextXofT)
        #print(thisYofT,nextYofT)
        Xangle = np.arctan(abs((nextXofT[0]-thisXofT[0]) / (1 + nextXofT[0]*thisXofT[0]))) 
        Xcos = np.cos(Xangle)
        Yangle = np.arctan(abs((nextYofT[0]-thisYofT[0]) / (1 + nextYofT[0]*thisYofT[0]))) 
        Ycos = np.cos(Yangle)
        print("X cosine score", Xcos)
        print("Y cosine score", Ycos)



In [ ]:

E = eList[500]


        
t0 = time.time()
compareImgFromEllipse(E)



In [ ]:
E[0]

In [ ]:
print(type(img1))

In [ ]:
stage6tubes[6]

In [ ]:
tubes[3]


In [ ]:
#Creating a dictionary of information for each tube on the event list
eT = [e[-2] for e in EventList]
eT2= [e[-1] for e in EventList]
evTubes = set(eT + eT2)

evData = []
for tnum in evTubes:
    Ts = np.array([t[5] for t in tubeList if t[-1] == tnum])
    Xs = np.array([t[0] for t in tubeList if t[-1] == tnum])
    Ys = np.array([t[1] for t in tubeList if t[-1] == tnum])
    thisdict = {"tubenum": tnum, "Ts": Ts, "Xs": Xs, "Ys":Ys}
    evData.append(thisdict)

In [ ]:
newtubes = deepcopy(tubes)
for e in EventList:
    dict1 = [d for d in evData if d['tubenum']==e[-2]][0]
    dict2 = [d for d in evData if d['tubenum']==e[-1]][0]
    
    #Gather data from before the event...
    T1 = dict1['Ts'][np.where(dict1['Ts']<e[0]-1)]
    T2 = dict2['Ts'][np.where(dict2['Ts']<e[0]-1)]
    X1 = dict1['Xs'][np.where(dict1['Ts']<e[0]-1)]
    X2 = dict2['Xs'][np.where(dict2['Ts']<e[0]-1)]
    Y1 = dict1['Ys'][np.where(dict1['Ts']<e[0]-1)]
    Y2 = dict2['Ys'][np.where(dict2['Ts']<e[0]-1)]

    if len(T1)<10 or len(T2)<10: #not enough info to do best fit
        continue
    
    print(e[-2])
    print(e[-1])
    #Get best fit equations
    XofT1 = np.polyfit(T1,X1,1)
    YofT1 = np.polyfit(T1,Y1,1)
    XofT2 = np.polyfit(T2,X2,1)
    YofT2 = np.polyfit(T2,Y2,1)
    print("First x(t) =", XofT1[0],"t + ", XofT1[1])
    print("First y(t) =", YofT1[0],"t + ", YofT1[1])
    print("Second x(t) =",XofT2[0],"t + ", XofT2[1])
    print("Second y(t) =",YofT1[0],"t + ", YofT2[1])
    toAdd1 = []
    toAdd2 = []
    prev1 = [ell for ell in newtubes[e[-2]] if ell[0]==e[0]][0]
    prev2 = [ell for ell in newtubes[e[-1]] if ell[0]==e[0]][0]
    #next1 = [ell for ell in newtubes[e[-2]] if ell[0]==e[1]][0]
    #next2 = [ell for ell in newtubes[e[-1]] if ell[0]==e[1]-5][0]
    
    #Make ghost tubes based on the best fit equations
    for f in range(e[0],e[0]+60):
        X1 = XofT1[0] * f + XofT1[1]
        Y1 = YofT1[0] * f + YofT1[1]
        smax1 = prev1[1][1][0]
        bgax1 = prev1[1][1][1]
        phi1 = prev1[1][2]
        Vx1 = XofT1[0]
        Vy1 = YofT1[0]
        RGB = [0,0,255]
        newe1 = [f, ((X1,Y1), (smax1,bgax1), phi1), [Vx1,Vy1], RGB]
        toAdd1.append(newe1)
        if f == e[0]:
            print(newe1)
            
        
        X2 = XofT2[0] * f + XofT2[1]
        Y2 = YofT2[0] * f + YofT2[1]
        smax2 = prev2[1][1][0]
        bgax2 = prev2[1][1][1]
        phi2 = prev2[1][2]
        Vx2 = XofT2[0]
        Vy2 = YofT2[0]
        RGB = [0,0,255]
        newe2 = [f, ((X2,Y2), (smax2,bgax2), phi2), [Vx2,Vy2], RGB]
        toAdd2.append(newe2)
        
    print()
    newtubes[e[-2]].extend(toAdd1)
    newtubes[e[-1]].extend(toAdd2)
    
        


In [ ]:
stage8tubes = newtubes
putTubesOnVid(newtubes, vidbase+'_outstage8quad_last30.mp4')

In [ ]:
newtubes = deepcopy(tubes)
for e in EventList:
    dict1 = [d for d in evData if d['tubenum']==e[-2]][0]
    dict2 = [d for d in evData if d['tubenum']==e[-1]][0]
    
    #Gather data from before the event...
    T1 = dict1['Ts'][np.where(dict1['Ts']<e[0])]
    T2 = dict2['Ts'][np.where(dict2['Ts']<e[0])]
    X1 = dict1['Xs'][np.where(dict1['Ts']<e[0])]
    X2 = dict2['Xs'][np.where(dict2['Ts']<e[0])]
    Y1 = dict1['Ys'][np.where(dict1['Ts']<e[0])]
    Y2 = dict2['Ys'][np.where(dict2['Ts']<e[0])]

    if len(T1)<10 or len(T2)<10: #not enough info to do best fit
        continue
    
    print(e[-2])
    print(e[-1])
    #Get best fit equations
    XofT1 = np.polyfit(T1[-30:],X1[-30:],2)
    YofT1 = np.polyfit(T1[-30:],Y1[-30:],2)
    XofT2 = np.polyfit(T2[-30:],X2[-30:],2)
    YofT2 = np.polyfit(T2[-30:],Y2[-30:],2)
    print("First x(t) =", XofT1[0],"t^2 + ", XofT1[1],"t", + XofT1[2])
    print("First y(t) =", YofT1[0],"t^2 + ", YofT1[1],"t", + YofT1[2])
    print("Second x(t) =",XofT2[0],"t^2 + ", XofT2[1],"t", + XofT2[2])
    print("Second y(t) =",YofT2[0],"t^2 + ", YofT2[1],"t", + YofT2[2])
    print()
    toAdd1 = []
    toAdd2 = []
    prev1 = [ell for ell in newtubes[e[-2]] if ell[0]==e[0]-10][0]
    prev2 = [ell for ell in newtubes[e[-1]] if ell[0]==e[0]-10][0]

    #Make ghost tubes based on the best fit equations
    for f in range(e[0],e[0]+60):
        X1 = (XofT1[0] * f**2) + (XofT1[1]*f) + XofT1[2]
        Y1 = (YofT1[0] * f**2) + (YofT1[1]*f) + YofT1[2]
        smax1 = prev1[1][1][0]
        bgax1 = prev1[1][1][1]
        phi1 = prev1[1][2]
        Vx1 = XofT1[0]
        Vy1 = YofT1[0]
        RGB = [0,0,255]
        newe1 = [f, ((X1,Y1), (smax1,bgax1), phi1), [Vx1,Vy1], RGB]
        toAdd1.append(newe1)
        if f == e[0]:
            print(newe1)
        
        X2 = (XofT2[0] * f**2) + (XofT2[1]*f) + XofT2[2]
        Y2 = (YofT2[0] * f**2) + (YofT2[1]*f) + YofT2[2]
        smax2 = prev2[1][1][0]
        bgax2 = prev2[1][1][1]
        phi2 = prev2[1][2]
        Vx2 = XofT2[0]
        Vy2 = YofT2[0]
        RGB = [0,0,255]
        newe2 = [f, ((X2,Y2), (smax2,bgax2), phi2), [Vx2,Vy2], RGB]
        toAdd2.append(newe2)
    
    newtubes[e[-2]].extend(toAdd1)
    newtubes[e[-1]].extend(toAdd2)
    
        


In [ ]:
print(T1[-30:])

In [ ]:
import cv2
cv2.__version__